# Name Entity Recognition

In [ ]:
! git clone https://github.com/EliaFeltrin/KIND_project

# TO DO In the introduction we have to put the introduction, the initial 
# consideration and so on, Elia's code has to be put here

In [ ]:
# General
! pip install pandas
! pip install tqdm
from tqdm import tqdm
import string
import numpy as np
import random
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt

In [ ]:
# Clustering
! pip install umap-learn
! pip install sklearn
! pip install nltk

from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.manifold import TSNE
import umap
from sklearn.feature_extraction.text import TfidfVectorizer

# embeddings for the clustering

!pip install -U sentence-transformers
! pip install plotly
import plotly
import plotly.graph_objects as go

from sentence_transformers import SentenceTransformer



In [ ]:
! pip install sklearn
from sklearn.decomposition import TruncatedSVD

import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import LabelEncoder


from sklearn.cluster import MiniBatchKMeans
from sklearn import metrics

In [ ]:
# CRF
!pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite
!pip install eli5

from sklearn_crfsuite import metrics as crf_metrics
from sklearn_crfsuite import CRF
import eli5

In [ ]:
# embeddings
! pip install --upgrade gensim
! pip install fasttext
from gensim.models.word2vec import Word2Vec
import fasttext
import fasttext.util

In [ ]:
#spacy
! pip install -U spacy

## Introduction

### Package Installation

In [ ]:
# Installing the library needed in the following part of the project

# Installing the utlity packages
#! pip install scipy
#! pip install numpy
! pip install pandas

# Installing a package for having the progress bar
!pip install tqdm

# Installing a natural language processing package
! pip install nltk

# Installing the packages for creating amazing plots
#! pip install matplotlib
! pip install wordcloud
! pip install plotly
! pip install --upgrade nbformat

# Installing a package for sequence labeling, used for POS tagging and NER
! pip install -U spacy

# Installing the packages for creating the word embeddings
! pip install --upgrade gensim
! pip install fasttext

# Installing one of the main machine learning libraries
! pip install sklearn

# Installing the packages for doing dimensionality reduction
! pip install umap-learn

# Installing packages for indexing the dataset
! pip install python-terrier

# Installing packages implementing conditional random fields
! pip install git+https://github.com/MeMartijn/updated-sklearn-crfsuite.git#egg=sklearn_crfsuite
! pip install eli5

# Installing packages for transformers
! pip install transformers==4.28.0
! pip install datasets
! pip install evaluate
! pip install seqeval

### Models download

In [ ]:
# Dowloading an italian model from spacy
! spacy download it_core_news_lg

### Package import

In [ ]:
# Importing the main packages

# Importing the utlity packages
import string
import numpy as np
import random
import pandas as pd
from collections import Counter

# Importing a package for having the progress bar
from tqdm import tqdm

# Importing a natural language processing library
import nltk

# Importing the packages for creating amazing plots
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import plotly.express as px

# Importing the packages for creating the word embeddings
from gensim.models.word2vec import Word2Vec
import fasttext
import fasttext.util

# Importing the packages for doing dimensionality reduction
from sklearn.manifold import TSNE
import umap

# Importing a package for the tf-idf representation of the sentences
from sklearn.feature_extraction.text import TfidfVectorizer

# Importing a package for clustering
from sklearn.cluster import KMeans
from sklearn import metrics

# Importing the packages for POS tagging
import spacy as spc
import it_core_news_lg

# Importing packages for indexing the dataset
import pyterrier as pt
if not pt.started():
  pt.init()

# Importing packages implementing conditional random fields
from sklearn_crfsuite import metrics as crf_met
from sklearn_crfsuite import CRF
import eli5

# Importing packages for Transformers
from datasets import Dataset, DatasetDict
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
from sklearn.model_selection import train_test_split
import torch
import evaluate
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## Data loading

In [ ]:
# Defining the names of the datasets
dataset_names_train_IO = ['degasperi_train.tsv', 'fiction_train.tsv', 'moro_train.tsv', 'wikinews_train.tsv']
dataset_names_test_IO = ['degasperi_test.tsv', 'fiction_test.tsv', 'moro_test.tsv', 'wikinews_test.tsv']
dataset_names_train_BIO = ['degasperi_train_BIO.tsv', 'fiction_train_BIO.tsv','moro_train_BIO.tsv', 'wikinews_train_BIO.tsv']
dataset_names_test_BIO = ['degasperi_test_BIO.tsv', 'fiction_test_BIO.tsv','moro_test_BIO.tsv', 'wikinews_test_BIO.tsv']

# Defining the path to datasets
PATH_TO_DATASETS_IO = '/content/KIND_project/datasets/Inside_outside_NER_notation'
PATH_TO_DATASETS_BIO = '/content/KIND_project/datasets/BIO_tag_NER_notation'

# Importing all the datasets in a dictionary
datasets_train_dict_IO = {name: pd.read_csv(PATH_TO_DATASETS_IO+'/'+name, sep='[\t|\n]', names=['Token', 'Entity'], engine='python') for name in dataset_names_train_IO}
datasets_test_dict_IO = {name: pd.read_csv(PATH_TO_DATASETS_IO+'/'+name, sep='[\t|\n]', names=['Token', 'Entity'], engine='python') for name in dataset_names_test_IO}
datasets_train_dict_BIO = {name: pd.read_csv(PATH_TO_DATASETS_BIO+'/'+name, sep='[\t|\n]', names=['Token', 'Entity'], engine='python') for name in dataset_names_train_BIO}
datasets_test_dict_BIO = {name: pd.read_csv(PATH_TO_DATASETS_BIO+'/'+name, sep='[\t|\n]', names=['Token', 'Entity'], engine='python') for name in dataset_names_test_BIO}

## Functions for dealing with the datasets

In this session we define some funtions useful for having the correct structure of the dataset in order to use in the various techniques we implement in the following part of the notebook.


In [ ]:
def get_string_from_df(dataframe_df, puntuaction, column_names=['Token', 'Entity']):
  '''
  Transforms the tokenized dataset into a single string.
  It is assumed that the dataframe has two columns: one column containing the 
  tokens, so the words of the various sentences, by default it is named 'Token',
  and the second containing the labels associated with that tokens, by default 
  it is named 'Entity'.

  Parameters
  ----------
  dataframe_df: DataFrame
    dataframe containing the tokenized dataset.
  puntuaction: list
    list containing the puntuaction, it is used in order to manage in a better
    way the spaces in case of an element of puntuaction.
  column_names: list, optional
    ordered names of the columns, the one with index zero is used for extracting
    the text from the DataFrame.

  Returns
  -------
  text: str
    string concatenating all the tokens of the dataset.

  '''

  text_df = dataframe_df.loc[:,column_names[0]]
  text = text_df[0]
  for token in text_df[1:]:
    text += (' ' + token) if token not in puntuaction else token
  return text

In [ ]:
def get_sentences_list_from_df(dataset_df, key=None, column_names=['Token', 'Entity'],
                               lower=False):
  '''
  Extracts and returns the sentences from the DataFrame given as input.
  It is assumed that the dataframe has two columns: one column containing the 
  tokens, so the words of the various sentences, by default it is named 'Token',
  and the second containing the labels associated with that tokens, by default 
  it is named 'Entity'.

  Parameters
  ----------
  dataset_df: DataFrame
    dataframe containing the tokenized dataset
  key: str, optional
    name of the dataset from which the dataframe comes from.
  column_names: list, optional
    ordered names of the columns, the one with index zero is used for extracting
    the text from the DataFrame.
  lower: bool, optional
    if True, the all the words will be transformed into lowercase words, if
    False, they will not.

  Returns
  -------
  sentences_list: list
    list of the sentences contained in the dataset.
  labels_list: list
    list of lists of labels associated to the words of the sentences contained 
    in the dataset.
  keys_list: list
    list containing an entry for each sentence with the name of the dataset it
    comes from.

  '''
  
  print('Converting the DataFrame of the dataset' + key if key != None else '' \
        + 'into a list of strings')
  
  punctuation = string.punctuation
  sentences_list = [[]]
  labels_list = [[]]
  keys_list = None
  count = 0

  for element in tqdm(dataset_df.iterrows()):
    if str(element[1]['Token']) == '.':
      sentences_list.append([])
      labels_list.append([])
      count += 1
    elif str(element[1]['Token']) not in punctuation:
      sentences_list[count].append(element[1]['Token'].lower() if lower else element[1]['Token'])
      labels_list[count].append(element[1]['Entity'])
  if key != None:
    keys_list = [key for sentence in range(len(sentences_list))]
  return sentences_list, labels_list, keys_list

################################################################################

def remove_short_sentences(sentences_list, labels_list, keys_list = None, min_length=3):
  '''
  Removes the sentences with a length lower than a certain threshold.

  Parameters
  ----------
  sentences_list: list
    list of the sentences to be analyzed.
  labels_list: list
    list of lists of labels associated to the sentences.
  keys_list: list, optional
    list containing the name of the dataset from which each sentence.
  min_lenght: int, optional
    minimum number of words a sentence has to have to be kept.

  Returns
  -------
  new_sentences_list: list
    list of the sentences contained in the dataset.
  new_labels_list: list
    list of lists of labels associated to the words of the sentences contained 
    in the dataset.
  new_keys_list: list
    list containing an entry for each sentence with the name of the dataset it
    comes from.

  '''

  print('Removing short sentences from the dataset')
    
  new_sentences_list = list()
  new_labels_list = list()
  new_keys_list = list()
  for idx in tqdm(range(len(sentences_list))):
    if len(sentences_list[idx]) >= min_length:
      new_sentences_list.append(sentences_list[idx])
      new_labels_list.append(labels_list[idx])
      if keys_list != None:
        new_keys_list.append(keys_list[idx])
  return new_sentences_list, new_labels_list, new_keys_list

################################################################################

def get_all_sentences_from_datasets(datasets, lower=False):
  '''
  Extracts and returns the sentences from every DataFrame in the dictionary
  given as input.

  Parameters
  ----------
  datasets: dict
    dictionary of DataFrames containing the tokenized datasets.
  lower: bool, optional
    if True, the all the words will be transformed into lowercase words, if
    False, they will not.
  
  Returns
  -------
  overall_sentences: list
    list of the sentences contained in the datasets given in input.
  overall_labels: list
    list of lists of labels associated to the words of the sentences contained 
    in the datasets.
  overall_keys: list
    list containing an entry for each sentence with the name of the dataset it
    comes from.

  '''

  overall_sentences = list()
  overall_labels = list()
  overall_keys = list()
  for key in datasets.keys():
    sentences, labels, keys = get_sentences_list_from_df(datasets[key], key=key, lower=lower)
    sentences, labels, keys = remove_short_sentences(sentences, labels, keys_list=keys)
    overall_sentences += sentences
    overall_labels += labels
    overall_keys += keys
  return overall_sentences, overall_labels, overall_keys

In [ ]:
def remove_stopwords(sentences, labels, stopwords):
  '''
  Removes the stopwords from the dataset updating the label list.

  Parameters
  ----------
  sentences: list
    list of the tokenized sentences.
  labels: list
    list of lists of labels associated to the sentences.
  stopwords: list
    list of stopwords.

  Returns
  -------
  overall_sentences: list
    list of the sentences contained in the datasets given in input.
  overall_labels: list
    list of lists of labels associated to the words of the sentences contained 
    in the datasets.

  '''

  sentences_no_stopwords = []
  labels_no_stopwords = []

  for i,sentence in enumerate(sentences):
    new_sentence = []
    new_labels = []
    for j,word in enumerate(sentence):
      if word not in stopwords:
        new_sentence.append(word)
        new_labels.append(labels[i][j])
    
    sentences_no_stopwords.append(new_sentence)
    labels_no_stopwords.append(new_labels)
  return sentences_no_stopwords, labels_no_stopwords

In [ ]:
# Defining some function useful for having the correct structure of the dataset
# in order to define the tf-idf representation

def concatenate_sentences_tokens(sentences):
  '''
  Recombines the tokenized senteces into complete strings concatenating the
  tokens of each sentence.

  Parameters
  ----------
  sentences: list
    list of tokenized sentences.

  Returns
  -------
  concatenated_sentences: list
    list of sentences.

  '''
  
  concatenated_sentences = list()
  for sentence in sentences:
    new_sentence = ''
    for token in sentence:
      new_sentence += (token + ' ')
    concatenated_sentences.append(new_sentence)
  return concatenated_sentences

In [ ]:
def get_datasets_names():
  '''
  Returns a list containing the names of the datasets
  
  '''
  
  return ['degasperi', 'fiction', 'moro', 'wikinews']

In [ ]:
def get_numerical_mapping():
  '''
  Returns a dictionary containing the mapping between the names of the datasets 
  and a numerical value

  '''

  return {'degasperi': 0, 'fiction': 1, 'moro': 2, 'wikinews': 3}

## Functions for doing some plots
In this session we define some funtions useful for creating 3D plots.

In [ ]:
def plot_3D(x, y, z, labels, mapping=None, title=''):
  '''
  Makes the 3D plot of some vectors using different colors for different classes
  of labels.

  Parameters
  ----------
  x: ndarray
    coordinates on x of the points.
  y: ndarray
    coordinates on x of the points.
  z: ndarray
    coordinates on x of the points.
  labels: list
    list of labels, one label for each point.
  mapping: dict
    mapping between each label and numerical value.
  title: str
    title of the plot.

  '''

  # Mapping the textual labels of the datasets to a number each in order to 
  # display them using different colors
  label_mapping = {}
  inverse_label_mapping = {}
  if mapping == None:
    for i, label in enumerate(list(set(labels))):
      label_mapping[label] = i
      inverse_label_mapping[i] = label
  else:
    label_mapping = mapping
    for key in label_mapping:
      value = label_mapping[key]
      inverse_label_mapping[value] = key
  numeric_labels = [label_mapping[label] for label in labels]

  # Creating an empty list to store the traces
  traces = []
  # Iterating over the numbers labelling the datasets and creating the traces of
  # their points
  for numeric_label in np.unique(numeric_labels):
    # Keeping only the points having the label considered in the iteration
    label_indices = np.where(numeric_labels == numeric_label)[0]
    x_label = x[label_indices]
    y_label = y[label_indices]
    z_label = z[label_indices]

    # Creating a trace for the 3D scatter plot
    trace = go.Scatter3d(
        x=x_label,
        y=y_label,
        z=z_label,
        mode='markers',
        marker=dict(
            size=3,
            color=numeric_label,
            opacity=0.9
        ),
        name=str(inverse_label_mapping[numeric_label]),
        #text=labels[inverse_label_mapping[numeric_label]],
        #legendgroup=labels[inverse_label_mapping[numeric_label]]
    )
    traces.append(trace)

  # Creating the layout
  layout = go.Layout(
      scene=dict(
          xaxis=dict(title='x'),
          yaxis=dict(title='y'),
          zaxis=dict(title='z')
      ),
      title=title,
      showlegend=True
  )

  # Creating the figure and add the trace
  fig = go.Figure(data=traces, layout=layout)

  # Enabling scene manipulation
  fig.update_layout(scene=dict(camera=dict(eye=dict(x=1, y=1, z=1))))

  # Showing the plot
  fig.show()

## Functions for computing some metrics

In this session we define some funtions useful for computing the main performance indeces usually avaluated for understanding the goodness of a classificator.

In [ ]:
def metrics(predictions, true_labels, metrics_df=None, class_label='NotLabeled'):
  '''
  Computes and print metrics TP, TN, FP, FN, AC, RC, PC, F1

  Parameters
  ----------
  predictions: ndarray
    predictions of samples obtained with a model
  true_labels: ndarray
    true labels of the samples
  metrics_df: DataFrame, optional
    DataFrame to which the computed statistics have to be put
  class_label: str, optional
    label identifing the belonging of the statistcs to its class

  Returns
  -------
  metrics_df: DataFrame
    DataFrame containing the statistics contained in the parameter metrics_df
    plus the statistics computed on the new predictions

  '''

  TP = np.sum(np.logical_and(predictions == 1., true_labels == 1.)) # Attacks accurately flagged as attacks
  TN = np.sum(np.logical_and(predictions == 0., true_labels == 0.)) # Normal traffic accurately flagged as normal
  FP = np.sum(np.logical_and(predictions == 1., true_labels == 0.)) # Normal traffic incorrectly flagged as attacks
  FN = np.sum(np.logical_and(predictions == 0., true_labels == 1.)) # Attacks incorrectly flagged as normal 

  AC = ((TN + TP) / len(predictions)) * 100 # Accuracy
  RC = (TP / (FN + TP)) * 100 # Recall or sensitivity
  PR = (TP / (FP + TP)) * 100 # Precision
  F1 = 2 * PR * RC / (PR + RC) # F1-Score

  if metrics_df is None:
    columns = ['Set of features', 'TP', 'TN', 'FP', 'FN', 'accuracy', 'recall', 'precision', 'F1-score']
    metrics_df = pd.DataFrame([[class_label, TP, TN, FP, FN, AC, RC, PR, F1]], columns=columns)
  else:
    columns = ['Set of features', 'TP', 'TN', 'FP', 'FN', 'accuracy', 'recall', 'precision', 'F1-score']
    metrics_df = pd.concat([metrics_df,pd.DataFrame([[class_label, TP, TN, FP, FN, AC, RC, PR, F1]], columns=columns)])
  
  return metrics_df

################################################################################

def performance_metrics(predicted_labels, real_labels):
  '''
  Computes performance metrics given the predictions and the true labels

  Parameters
  ----------
  predictions: ndarray
    predictions of samples obtained with a model
  true_labels: ndarray
    true labels of the samples

  Returns
  -------
  metrics_df: DataFrame
    DataFrame containing the statistics contained in the parameter metrics_df
    plus the statistics computed on the new predictions

  '''
  
  metrics_df = None
  classes = set(element for sentence in real_labels for element in sentence)
  for tag in classes:
    new_predicted_labels = np.array([1 if element == tag else 0 for sentence in predicted_labels for element in sentence])
    new_real_labels = np.array([1 if element == tag else 0 for sentence in real_labels for element in sentence])
    metrics_df = metrics(new_predicted_labels, new_real_labels, metrics_df=metrics_df, class_label=tag)
  
  return metrics_df

## Word embeddings representation

A word embedding is the vectorial representation of a word that contains its sintactic and semantic information. It is used for achieving a numerical and dense representation of the words in an high dimensional space. Other types of representation of the words, such as the bag-of-words one, are sparse compared to word embeddings.
The word embeddings allow to achieve better results in many fields of natural language processing.

In this section, we train a Word2Vec model in order to obtain the word embeddings of the words contained in the dataset and we also load a model from FastText and find the representation of the dataset in terms of its vectors.

The basic version of the word embeddings model is a artificial neural network composed by two layers.

The inputs of the neural network are the words of the sentence to convert in the word embeddings representation.
The first layer of the neural network is a fully connected linear layer with a number of neurons equals to the dimension of the embeddings.
This layer is used to feed the last layer of the neural network that has the same number of outputs as inputs. On the output layer is applied a softmax function.
The neural network is then trained in order to make it predict, for each word in input, the next word in a sentence.
This is done, in the basic implementation by setting the input corresponding to the considered word to 1 and the other inputs to 0 and expecting as result the probability 1 on the expected word (the output of the softmax layer can be interpreted as a probability).
The training can be performed using the cross entropy as loss function. At the end of the training the weights connecting each input to the first hidden layer are the values of the dimensions of the word embedding corresponding to the input word.

Two of the most used architecture of the Word2Vec embedding model are CBOW and Skip-Gram.

The Continuous Bag of Words (CBOW) method uses many words surrounding the considered word for predicting it. In the training step, the model sets to 1 the inputs corresponding to the words in the context of a word, the neural network has to predict the word inside the context. In this way the embedding is used to embed the context of a word for representing it.
The Skip-gram uses a word to predict the words in the surroundings. In the training step, the input of the considered word will be set to 1 and the outputs the neural network has to learn are the probabilities of the words in its context, so the probabilities of the words in the surroundings.

There are then optimizations of the training such as the negative sampling method or the computation of only some selected and meaningful outputs for each word in the train set.

In some way the distribution in the various dimension is based on the similarity of the words in terms of semantics and usage. With word embeddings we can embed the context of the word inside its representation.


### Word embeddings trained on all the data

The choosen pipeline for the creation of the embeddings starts from the dataframe containing the tokens of the datasets.

The preprocessing applied to the datasets in order to define the input of the Word2vec model is the following:
- tokenization of the dataset (previously done by the providers of the datasets);
- transformation of the input DataFrame into a list of lists of tokens, this is done merging the elements of the datasets into sentences. Since the datasets did not contain specific separator for the sentences, it is assumed that a sentence ends when in the DataFrane there is a dot. Therefor, the split in sentences of the tokens is done basing on the single dots.
- lowercasing all the tokens

We choose to try the definition of the word embeddings model on different portions of the dataset in order to compare the results. We expect that the more data are considered in the model definition, the more good will be the model in terms of information catched inside it.

Firstly we try to define the word embeddings on the entire data we have since, as it is already been said, the larger is the dataset the better will be the our model and the wider will be the dictionary.

In [ ]:
# Merging the dictionaries of the trainsets and testsets
datasets_for_embeddings = datasets_train_dict_IO.copy()
datasets_for_embeddings.update(datasets_test_dict_IO)
# Merging the lists of the names of the trainsets and testsets
dataset_names_for_embeddings = dataset_names_train_IO + dataset_names_test_IO

# Getting the sentences, the labels and the origin dataset of the sentences in
# the correct format in order to create the embeddings representation of the words
sentences, labels, sentences_keys = get_all_sentences_from_datasets(datasets_for_embeddings)

# Printing 10 lists of token
print()
print('Here some examples of the shape of the sentences:')
for i in range(10):
  print(str(sentences[i]))

In [ ]:
# To check that there are no tabs or new lines inside the tokens
# It was used because there were issues with the reading of the datasets
found = False
for name in dataset_names_for_embeddings:
  for i, el in datasets_for_embeddings[name].iterrows():
    if '\t' in el['Token'] or '\n' in el['Token']:
      found = True
      print(el['Token'])
if found:
  print('There is something wrong, there tab or new line characters, are check the import of the dataset :(')
else:
  print('No tab or new line characters found, Great job!')

In [ ]:
# Printing some information about the list of lists of token
print('The total number of sentences in the dataset is ' + str(len(sentences)))
length_list= []
for idx,i in enumerate(sentences):
  length_list.append(len(i))
print('The maximum lenght of a sentence is ' + str(max(length_list)))

In [ ]:
# Definition of the Word2Vec model
embeddings_model = Word2Vec(sentences, vector_size=30, min_count=2, window=20)
# Printing the length of the vocabulary
print('The length of the vocabulary is ' + str(len(embeddings_model.wv)) + ' words')

In [ ]:
# Searching the most similar word to a specific word
term = 'governo'
embeddings_model.wv.most_similar(term.lower())

In [ ]:
# Definition of a function to get the word2vec vector representation of a word
def get_word2vec_vector(word, model):
  '''
  Returns the word2vec vector representation of a word.
  
  Parameters
  ----------
  word: str
    word to convert into its vectorial representation.
  model: Word2Vec
    word2Vec model from gensim library for creating the embedding.

  Returns
  -------
  word_vector: ndarray
    word2Vec vector representation of a word.

  '''
  
  word_vector = np.zeros(model.vector_size)
  try:
    word_vector = model.wv[word]
  except KeyError:
    try:
      word_vector = model.wv[model.wv.most_similar(word)[0][0]]
    except:
      return word_vector
    return word_vector
  return word_vector

Here we can try to find the most similar words to the given one. Using different values we can understand that the model is quite good and retrives words very related to the selected one.

In [ ]:
# Choosing a subset of embedding vectors
word_samples = random.sample(list(embeddings_model.wv.key_to_index), 500)
word_vectors = embeddings_model.wv[word_samples]

# Computing the dimensionality reduction of the word embeddings space
tsne = TSNE(n_components=3, n_iter=2000)
tsne_embeddings = tsne.fit_transform(word_vectors)
x, y, z = np.transpose(tsne_embeddings)

In [ ]:
# Plotting the word embeddings of the model
fig = px.scatter_3d(x=x, y=y, z=z, text=word_samples)
fig.update_traces(marker=dict(size=3, line=dict(width=2)), textfont_size=6)
fig.show()

The second analysis is done on the word embeddings trained only on the training datasets that could be useful in the next part of the project since they can be used for performing named entity recognition. 

This is indeed a more realistic scenario, since train set and test set must be kept separate as much as possible in order to have an unbiased assessment of the models when evaluating it on the test set.

In [ ]:
# Getting the sentences, the labels and the origin dataset of the sentences in
# the correct format in order to create the embeddings representation of the words
sentences_train, labels_train, sentences_keys_train = get_all_sentences_from_datasets(datasets_train_dict_IO, lower=False)

# Definition of the Word2Vec model
embeddings_model_train = Word2Vec(sentences_train, vector_size=50, min_count=2, window=10)

In [ ]:
# Printing the length of the vocabulary
print('The length of the vocabulary is ' + str(len(embeddings_model_train.wv)) + ' words')

In [ ]:
# Searching the most similar word to a specific word
term = 'governo'
embeddings_model_train.wv.most_similar(term.lower())

In [ ]:
# Choosing a subset of embedding vectors
word_samples_train = random.sample(list(embeddings_model_train.wv.key_to_index), 500)
word_vectors_train = embeddings_model_train.wv[word_samples_train]

# Computing the dimensionality reduction of the word embeddings space
tsne = TSNE(n_components=3, n_iter=2000)
tsne_embeddings = tsne.fit_transform(word_vectors_train)
x_train, y_train, z_train = np.transpose(tsne_embeddings)

In [ ]:
# Plotting the word embeddings of the model
fig = px.scatter_3d(x=x_train, y=y_train, z=z_train, text=word_samples_train)
fig.update_traces(marker=dict(size=3, line=dict(width=2)), textfont_size=6)
fig.show()

The dictionary is a bit smaller but still big and the results are quite similar to the previous ones, in terms of understanding and search of the similar words and also the embedding space has a similar shape. 

Since train set and test set are usually separate and so I could create the embeddings model for the train set first and then model inside it the test set.

However it has to be noted that the vocabulary is a bit smaller with respect to the one of the model trained over all the data. This is believable since the test set can have token that the train set doesn't present. This aspect could be a problem when the models fed by the embeddings have to be applied on the representation of the test set. Some words will not be converted in embeddings and the that words will could not be used.


The last embedding models trained are the ones computed on the train datasets separately.

In [ ]:
# Separating the train sets and collecting their sentences, labels and origin
# dataset into a dictionary
sentences_train_separate = {}
labels_train_separate = {}
embeddings_model_train_separate = {}
keys_train_separate = {}
for name in dataset_names_train_IO:
  # Getting the sentences, the labels and the origin dataset of the sentences in
  # the correct format in order to create the embeddings representation of the words
  sentences_train_sep, labels_train_sep, keys_train_sep = get_all_sentences_from_datasets({name:datasets_train_dict_IO[name]})
  sentences_train_separate[name] = sentences_train_sep
  labels_train_separate[name] = labels_train_sep
  keys_train_separate[name]  = keys_train_sep 

  # Definition of the Word2Vec models
  embeddings_model_train_separate[name] = Word2Vec(sentences_train_sep, vector_size=30, min_count=2, window=20)

In [ ]:
# Printing the length of the vocabularies of the various datasets
for name in dataset_names_train_IO:
    print('The dictionary of the dataset ' + name + ' is long ' + str(len(embeddings_model_train_separate[name].wv)) + ' words')

In [ ]:
# Searching the most similar word to a specific word
for name in dataset_names_train_IO:
    term = 'governo'
    print(name + ':' + str(embeddings_model_train_separate[name].wv.most_similar(term.lower())))

It is easy to understand that the dictionaries in this case are smaller since the overall words are splitted and analysed in many models.

Trying to search the most similar embeddings to a given word in many different fields and for the different datasets, it can be seen that better performace are achieved by the datasets that are specialized in that field, e.g. fiction_train finds worst results (less related word embeddings) for the word 'governo' than the other datasets, which deal with news and politics.

In [ ]:
points_train_separate = {}
word_samples_train_separate = {}
for key in embeddings_model_train_separate:
  # Choosing a subset of embedding vectors
  word_samples_train_separate[key] = random.sample(list(embeddings_model_train_separate[key].wv.key_to_index), 500)
  word_vector_train_separate = embeddings_model_train_separate[key].wv[word_samples_train_separate]

  # Computing the dimensionality reduction of the word embeddings space
  tsne = TSNE(n_components=3, n_iter=1000)
  tsne_embeddings = tsne.fit_transform(word_vector_train_separate)
  points_train_separate[key] = np.transpose(tsne_embeddings)

In [ ]:
# Plotting the word embeddings of the model
for key in points_train_separate:
  fig = px.scatter_3d(x=points_train_separate[key][0], y=points_train_separate[key][1],
                      z=points_train_separate[key][2], text=word_samples_train_separate[key])
  fig.update_traces(marker=dict(size=3, line=dict(width=2)), textfont_size=6)
  fig.show()

### Computing the embeddings using a pretrained model FastText

In [ ]:
# Downloading the pretrained model from the fasttext library
!wget http://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.it.300.bin.gz
!gzip -d cc.it.300.bin.gz

In [ ]:
# Instantiating the model
ft_embedding_model = fasttext.load_model('cc.it.300.bin')

In [ ]:
# Printing some statistics about the model
print('The size of the vocabulary is ' + str(len(ft_embedding_model.get_words())))
print('The length of the embeddings is '+ str(ft_embedding_model.get_dimension()))

In [ ]:
# Searching the most similar word to a specific word
term = 'legge'
ft_embedding_model.get_nearest_neighbors(term.lower())

In [ ]:
def get_fasttext_vector(word, model=None):
  '''
  Returns the fastText vector representation of a word.
  
  Parameters
  ----------
  word: str
    word to convert into its vectorial representation.
  model: fasttext.FastText._FastText
    fastText model for creating the embedding. Providing it the computation of 
    the embedding is faster.

  Returns
  -------
  word_vector: ndarray
    fastText vector representation of a word.

  '''
  
  if model == None:
    model = fasttext.load_model('cc.it.300.bin')
  word_vector = model.get_word_vector(ft_embedding_model.get_nearest_neighbors(word.lower())[0][1])
  return word_vector

In [ ]:
# Finding the embedding representation of a word, assuming to represent a word
# with its similar word in the model
term = 'legge'
get_fasttext_vector(term, ft_embedding_model)

The following snippets of code try to transform the whole dataset in its fastText embedding representation. The time required to run this code is ....

In [ ]:
# Creating the embedding representation of the dataset
#ft_embedding_train = [[get_fasttext_vector(sentences_train[i][j], ft_embedding_model) for j in range(len(sentences_train[i]))] for i in range(len(sentences_train))]
# This operation takes a very long time to be computed but when it is done the 
# embeddings can be saved and loaded whenever they are needed

In [ ]:
# Computing the dimensionality reduction of the word embeddings space
#sentence_samples_ft_train = random.sample(ft_embedding_train, 200)
#word_samples_ft_train = random.sample(sentence_samples_ft_train, 2)
#tsne = TSNE(n_components=3, n_iter=2000)
#tsne_embeddings = tsne.fit_transform(word_samples_ft_train)
#x_ft_train, y_ft_train, z_ft_train = np.transpose(tsne_embeddings)

In [ ]:
# Plotting the word embeddings of the model
#fig = px.scatter_3d(x=x_ft_train, y=y_ft_train, z=z_ft_train, text=word_samples_ft_train)
#fig.update_traces(marker=dict(size=3, line=dict(width=2)), textfont_size=6)
#fig.show()

## Clustering

The clustering is an unsupervided machine learning technique useful for inspecting non-labeled data in order to find inside them hidden patterns or other information.
In our case clustering is not needed for performing named entity recognition but it could useful for gain some additional knowledge about the dataset before proceeding in the sequence labeling task.


Given the KIND dataset it is not possible to identify the different documents because no document separators are present in the data. Since it is not possible to identify the various documents, we decide to apply the clustering techniqes on the sentences. As we did in the embeddings section, we assume that the sentences were separate by the dot and we extract in this way them from the input files.

In this section we try to investigate whether the four dataset are distinguishable basing on the representation of their sentences and so in some way they have different characteristics.

### Importing the italian stopwords

In [ ]:
# Importing the italian stopwords (taken from https://github.com/stopwords-iso/stopwords-it.git)
with open('/content/KIND_project/notebook/stopwords-it.txt', 'r') as f:
    italian_stopwords = f.read()
italian_stopwords_github = italian_stopwords.split('\n')
print('Number of stopwords in the repository https://github.com/stopwords-iso/stopwords-it.git: ' + str(len(italian_stopwords_github)))

In [ ]:
# Downloading and importing the italian stopwords in the package nltk
nltk.download('stopwords')
italian_stopwords_nltk = nltk.corpus.stopwords.words('italian')
print('Number of stopwords in the nltk library: ' + str(len(italian_stopwords_nltk)))

In [ ]:
# Checking that all the stopwords of nltk are in the other dataset, better result with more stopwords
count = 0
for i in italian_stopwords_nltk:
    if i in italian_stopwords_github:
        count += 1
print(str(count) + ' stopwords of the nltk library are in the Github repository')
italian_stopwords = italian_stopwords_github

In [ ]:
# Getting the sentences, the labels and the origin dataset of the sentences in
# the correct format in order to create the embeddings representation of the words
sentences, labels, sentences_keys = get_all_sentences_from_datasets(datasets_train_dict_BIO, lower=False)

# Defining the labels of the clustering
for i, key in enumerate(sentences_keys):
  sentences_keys[i] = key.replace('_train_BIO.tsv','').replace('_test_BIO.tsv', '')

### Clustering TF-IDF vectors

The Term Frequency-Inverse Document Frequency is a numerical statistic used to evaluate the importance of a term within a document or a corpus. In our case, we evaluate the importance of the words in the sentences.
The first step is to obtain a TF-IDF representation of the dataset so from  the collection of text we achieve a matrix representation containing the TF-IDF values of each word in each sentence.

In [ ]:
# Defining the vectorizer model
vectorizer = TfidfVectorizer(max_df=0.2, min_df=0, stop_words=italian_stopwords_github, use_idf=True)
# Fitting the vectorizer model
vectorizer.fit(concatenate_sentences_tokens(sentences))

# Given that we are evaluating sentences the frequency can be lower for saying that we have a stopword

In [ ]:
# Extracting the vocabulary
vocabulary = vectorizer.get_feature_names_out()

print('The vocabulary is long ' + str(len(vocabulary)) + ' words')

In [ ]:
# The words look reasonable and the vocabulary seems to not have many stopwords inside
sorted(random.sample(vocabulary.tolist(), 100))

In [ ]:
# Converting the strings into vectors
sentences_vector = vectorizer.transform(concatenate_sentences_tokens(sentences))

In [ ]:
sentences_vector[0].multiply(sentences_vector[0]).sum()

In [ ]:
max_score = 0
sentence_idx = 45
for i in range(sentences_vector.shape[0]):
    if i != sentence_idx:
        score = sentences_vector[sentence_idx].multiply(sentences_vector[i]).sum()
        if score > max_score:
            max_score = score
            max_score_idx = i
print('Sentence index ' + str(sentence_idx) + ': ' + str(sentences[sentence_idx]))
print('Most similar sentence is the one with index ' + str(max_score_idx) + ': ' + str(sentences[max_score_idx]))
print('The score is ' + str(max_score))

Trying to find the most similar sentences to a given sentence, it is possible to see that the metrics of similarity are quite low, very often in the interval (0.20, 0.40). Moreover the retrieved sentences have some sort of similarities, they have some words in common, but they are not very similar in the semantics or in the field of application of the sentence.

The second step is to actual cluster the TF-IDF representation of the sentences.
In general it is not known the number of clusters that are present. It is possible to approximately define a range or deduce the number from the problem.
In the considered case, the clustering is applied for checking whether the 4 inputs datasets are distinguishable in some way, so initially 4 cluster are considered.

The K-means clustering method is the one used in the following part section.

In [ ]:
num_clusters= len(set(sentences_keys))
# Defining the K-means model
kmeans_model = KMeans(n_clusters=num_clusters, max_iter=1000, n_init=3, verbose=True,
                      random_state=2307)
# Applying K-means clustering to the tf-idf representation of the sentences
kmeans_model.fit(sentences_vector)

In [ ]:
# Displaying the centroids in terms of their words with highest score
for i in range(num_clusters):
  # Getting the centroid for the cluster of index i
  centroid = kmeans_model.cluster_centers_[i]
  sorted_terms = centroid.argsort()[::-1]
  
  print('Centroid of cluster ' + str(i)+':')
  print([vocabulary[j] for j in sorted_terms[:20]])

Even if some difference in the words of the centroids of the clusters can be seen, it is evident that the centroids are not very different one from the others meaning that the clustering of the sentences in 4 clusters does not obtain very good results.




In [ ]:
# Printing the number of sentences contained in the various datasets
print('Number of sentences in the various datasets: ')
counter = Counter(sentences_keys)
for dataset_name, count in counter.items():
  print('Dataset ' + dataset_name + ': ' + str(count))

In [ ]:
# Printing the number of sentences contained in the various clusters
print('Number of sentences in: ')
for i in range(kmeans_model.n_clusters):
    print(f"Cluster {i}: {np.sum(kmeans_model.labels_ == i)}")

In [ ]:
# Evaluating the clustering performance using intrinsic evaluation measures.
print("Intrinsic evaluation measures:")
print("Within-cluster sum-of-squares:", str(kmeans_model.inertia_))
print("Silhouette coefficient:", str(metrics.silhouette_score(sentences_vector, kmeans_model.labels_)))
# These metrics assess how much the clusters are internally coherent and how much 
# they are different one with respect to the others

In [ ]:
# Evaluating the clustering performance using extrinsic evaluation measures
print('Extrinsic evaluation measures:')
print("Homogeneity:", str(metrics.homogeneity_score(sentences_keys, kmeans_model.labels_)))
print("Completeness:", str(metrics.completeness_score(sentences_keys, kmeans_model.labels_)))
print("V-measure:", str(metrics.v_measure_score(sentences_keys, kmeans_model.labels_)))
print("Adjusted Rand-Index:", str(metrics.adjusted_rand_score(sentences_keys, kmeans_model.labels_)))
# These metrics assess how good the clusters have classified the elements of the
# dataset with respect to the known labels

In [ ]:
# Applying a SVD dimensionality reduction to the embedding representation of the
# sentences, here it is a linear dimensionality reduction operator for computational
# performance reason
svd = TruncatedSVD(3)
reduced_data = svd.fit_transform(sentences_vector)

[x,y,z] = np.transpose(reduced_data)
[x,y,z]

In [ ]:
# Plotting the 3D-dimensionality reduction of the the embeddings coloring in 
# different ways data coming from different datasets
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c=LabelEncoder().fit_transform(sentences_keys), marker='.')

In [ ]:
# Plotting the 3D-dimensionality reduction of the the embeddings coloring in 
# different ways the elements of the different clusters
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, z, c=kmeans_model.labels_, marker='.')

In [ ]:
# Plotting, in interactive mode, the 3D-dimensionality reduction of the the 
# embeddings coloring in different ways data coming from different datasets
plot_3D(x, y, z, sentences_keys, mapping=get_numerical_mapping(), title='3D-dimensionality reduction of the dataset plotted by dataset')

In [ ]:
# Plotting, in interactive mode, the 3D-dimensionality reduction of the the 
# embeddings coloring in different ways the elements of the different clusters
plot_3D(x, y, z, kmeans_model.labels_, '3D-dimensionality reduction of the dataset plotted considering the clusters')

Using TF-IDF vectors it is not possible to distinguish the datasets, the clusters found are not the ones of the 

### Clustering embedding vectors

In the following section it is provided the application of the clustering to the embeddings representation af the dataset in order to see whether the results are better in this case or not.

In [ ]:
# Removing the italian stopwords from the dataset
#sentences, labels = remove_stopwords(sentences, labels, italian_stopwords)
# Not needed, the embeddings wants the context, transformers works well with them

In [ ]:
# Defining the pretrained BERT model used to create the embeddings starting from
# the sentences
sbert_model = SentenceTransformer('dbmdz/bert-base-italian-xxl-cased')
# Joining the tokens of the sentences in order to obtain a list of sentences
sentences_list = [' '.join(sentence) for sentence in sentences]
# Create the embeddings starting from the sentences
sentence_embeddings_bert = sbert_model.encode(sentences_list)

In [ ]:
# Defining the K-means model
kmeans_model_embeddings = KMeans(n_clusters=4, max_iter=2000, n_init=10, verbose=True, random_state=2307)
# Applying K-means clustering to the tf-idf representation of the sentences
kmeans_model_embeddings.fit(sentence_embeddings_bert)

In [ ]:
# Printing the number of sentences contained in the various datasets
print('Number of sentences in the various datasets: ')
counter = Counter(sentences_keys)
for dataset_name, count in counter.items():
  print('Dataset ' + dataset_name + ': ' + str(count))

In [ ]:
# Printing the number of sentences contained in the various clusters
print('Number of sentences in: ')
for i in range(kmeans_model_embeddings.n_clusters):
    print(f"Cluster {i}: {np.sum(kmeans_model_embeddings.labels_ == i)}")

In [ ]:
# Evaluating the clustering performance using intrinsic evaluation measures
print("Intrinsic evaluation measures:")
print("Within-cluster sum-of-squares:", str(kmeans_model_embeddings.inertia_))
print("Silhouette coefficient:", str(metrics.silhouette_score(sentence_embeddings_bert, kmeans_model_embeddings.labels_)))
# These metrics assess how much the clusters are internally coherent and how much 
# they are different one with respect to the others

In [ ]:
# Evaluating the clustering performance using extrinsic evaluation measures
print('Extrinsic evaluation measures:')
print("Homogeneity:", str(metrics.homogeneity_score(sentences_keys, kmeans_model_embeddings.labels_)))
print("Completeness:", str(metrics.completeness_score(sentences_keys, kmeans_model_embeddings.labels_)))
print("V-measure:", str(metrics.v_measure_score(sentences_keys, kmeans_model_embeddings.labels_)))
print("Adjusted Rand-Index:", str(metrics.adjusted_rand_score(sentences_keys, kmeans_model_embeddings.labels_)))
# These metrics assess how good the clusters have classified the elements of the
# dataset with respect to the known labels

In [ ]:
# Applying a SVD dimensionality reduction to the embedding representation of the
# sentences, here it is a linear dimensionality reduction operator for computational
# performance reason
svd = TruncatedSVD(3)
reduced_data = svd.fit_transform(sentence_embeddings_bert)

[x, y, z] = np.transpose(reduced_data)
[x, y, z]

In [ ]:
# Plotting the 3D-dimensionality reduction of the the embeddings coloring in 
# different ways data coming from different datasets
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, c=LabelEncoder().fit_transform(sentences_keys), marker='.')

In [ ]:
# Plotting the 3D-dimensionality reduction of the the embeddings coloring in 
# different ways the elements of the different clusters
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')
ax.scatter(x, y, c=kmeans_model_embeddings.labels_, marker='.')

In [ ]:
# Plotting, in interactive mode, the 3D-dimensionality reduction of the the 
# embeddings coloring in different ways data coming from different datasets
plot_3D(x, y, z, sentences_keys, mapping=get_numerical_mapping(), title='3D-dimensionality reduction of the dataset plotted by dataset')

In [ ]:
# Plotting, in interactive mode, the 3D-dimensionality reduction of the the 
# embeddings coloring in different ways the elements of the different clusters
plot_3D(x, y, z, kmeans_model_embeddings.labels_, title='3D-dimensionality reduction of the dataset plotted considering the clusters')

To summarize, it is possible to see that the dataset is not meant for appying clustering and we cannot state that the sentences in the various datasets are very distinguishable.

Here it is done a brief analysis about the number of clusters that could be find in the problem....

In [ ]:
# Applying multiple times the K-means with mini-batch using a different number
# of clusters in different iterations
performance = [MiniBatchKMeans(n_clusters=k, batch_size=500, n_init=2, random_state=2307).fit(sentence_embeddings_bert).inertia_ for k in range(1,50)]
performance

In [ ]:
# Plotting the performances for various values of the number of clusters
plt.figure()
plt.plot(performance[:15])
plt.ylabel('Within-cluster sum-of-squares')
plt.xlabel('k')
plt.grid()
plt.show()

It can be seen that the elbow is approximatively in the interval [4,7].

## POS tagging

**Part-of-speech** tagging is the process of marking up a word in a text with the corresponding part of speech, based on both its definition and its context.
The POS Tagging here plays a crucial role to understand in what context the word is used in the sentence.

We will perform POS tagging using the **spaCy** libary.
After the tokenization, spaCy can parse and tag a given Doc. This is made thanks to a trained pipeline and its statistical models, which enable spaCy to make predictions of which tag o label most likely applies in this context.


### Creation of the POS tags files

In the following part of code we created some files containing the POS tags for each dataset and also che combination of them. This to improve the performances for the next steps and to avoid every time to do everything from the start.

In [ ]:
def write_pos_tags_on_file(filename, dataset):

  '''
  Takes as input a dataset and writes into a file the pos tags for each sentence

  Parameters
  ----------
  filename: Str
    File name that has to be used to save the pos tags

  dataset: DataFrame
    Dataset that contains the tokens of the text.

  '''

  sentences, labels, _ = get_sentences_list_from_df(dataset, lower=False)
  pos_tags = get_pos_tags(sentences)
  with open(f'{filename}.txt', 'w+') as f:
    for line in pos_tags:
      f.write(' '.join(line)+'\n')
  f.close()

In [ ]:
# For every dataset it writes the respective file with the pos tags
for dataset_name, dataset in datasets_train_dict_BIO.items():
  write_pos_tags_on_file(f'POS_tags_{dataset_name[:-4]}', dataset)

In [ ]:
# Write the pos tags for the train and test datasets merged
df_merged_train = pd.concat(datasets_train_dict_BIO.values(), ignore_index=True)
df_merged_test = pd.concat(datasets_test_dict_BIO.values(), ignore_index=True)

write_pos_tags_on_file('trainsets', df_merged_train)
write_pos_tags_on_file('testsets', df_merged_test)

### Plot stats POS tags


In [ ]:
# Importing the model here for being sure the model is available
import it_core_news_lg

def get_pos_tags(sentences):
  
  '''
  Computes and returns the pos tags of the sentences of the dataset.

  Parameters
  ----------
  sentences: list
    list of lists of words that is the list of the tokenized sentences.

  Returns
  -------
  pos_tags: list
    list of lists of part-of-speech tags, in the same format of the input 
    sentences, there is a mapping one-to-one between words and tags.

  '''

  # Loading the model for POS tagging
  model = it_core_news_lg.load()
  
  # Join the tokens into sentences
  sentences_tokens_merged = [' '.join(sentence) for sentence in sentences]

  pos_tags = []
  for sentence in sentences_tokens_merged: 
    # Process the sentence with SpaCy
    doc = model(sentence)
    sentence_pos = []
    for token in doc:
      sentence_pos.append(token.pos_)
    pos_tags.append(sentence_pos)

  return pos_tags

In [ ]:
def get_pos_tags_file(train_or_test=True):
  '''
  Retrieves the part-of-speech tags of the dataset in a format easy to use in
  our analysis.

  Parameters
  ----------
  train_or_test: bool, optional
    if True, the method returns the tags of the train set, if False, the method 
    returns the tags of the test set.

  Returns
  -------
  pos_tags: list
    part-of-speech tags of the requested dataset.

  '''

  with open('/content/KIND_project/pos_tags/POS_tags_'+('train' if train_or_test else 'test')+'sets.txt', 'r') as f:
    pos_tags = [line.replace('\n', '').split(' ') for line in f]
    return pos_tags
  f.close()

def get_pos_tags_specific_file(dataset_name):

  '''
  Retrieves the part-of-speech tags of a specific dataset in a format easy to use in our analysis.

  Parameters
  ----------
  file_name: Str
    Name of the dataset

  Returns
  -------
  pos_tags: list
    part-of-speech tags of the requested dataset.

  '''

  with open(f'/content/KIND_project/pos_tags/POS_tags_{dataset_name}.txt', 'r') as f:
    pos_tags = [line.replace('\n', '').split(' ') for line in f]
    return pos_tags
  f.close()

In [ ]:
def merge_counters(counter1, counter2):

  '''
  Takes 2 counters with different shapes and in the smallest one ad also the key that are currently inside with a value of 0

  Parameters
  ----------
  counter1: Counter
    Counter with smallest shape that has to be incremented
  counter2: Counter
    Counter with the larger shape

  Returns
  -------
  new_counter: Counter
    Counter containing all the tuple of the smallest one and the tuples (key, 0) of the larger one that were no present in the small one
  
  '''
  
  new_counter = counter1    

  for key, value in counter2.items():
    if key not in new_counter.keys():
      new_counter[key] = 0 

  return new_counter

In [ ]:
def plot_wordCloud_counters(counters):

  '''
  Takes as input a list of counters and it plots the wordCloud

  Parameters
  ----------
  cunters: list(Counter)
    List of counters that has to be plotted. It does not require that all the counters has the same shape

  '''

  word_cloud_counter = Counter()
  for counter in list_counters:
    word_cloud_counter.update(counter)
  # Generate a word cloud from the POS counts
  wordcloud = WordCloud(background_color='white').generate_from_frequencies(word_cloud_counter)

  # Plot the word cloud
  plt.imshow(wordcloud, interpolation='bilinear')
  plt.axis("off")
  plt.show()

In [ ]:
def plot_groupedBar_counters(counters):

  '''
  Takes as input a list of counters and it plots in the same bar chart the counts

  Parameters
  ----------
  cunters: list(Counter)
    List of counters that has to be plotted. It does not require that all the counters has the same shape

  '''
  
  max_length = max(map(len, list_counters))                         # max length of the counters
  max_position = list(map(len, list_counters)).index(max_length)    # position in the list of the counter with max length

  # For each counter that is not the one of maximum dimension I merge it with all the other ones. The result is a list with counters having all the same keys
  for i in range(len(list_counters)):
    if i != max_position:
      list_counters[i] = merge_counters(list_counters[i], list_counters[max_position])

  # We plot each counter inside the bar chart
  x = np.arange(max_length)
  width=0.2
  multiplier = 0
  for counter in list_counters:
    offset = width * multiplier
    labels, values = zip(*sorted(counter.items()))
    plt.bar(x + offset, values, width=width)
    multiplier += 1

  plt.title("POS Tag Frequency Distribution for datasets")
  plt.xlabel("POS Tag")
  plt.ylabel("Frequency")
  plt.xticks(x + width, sorted(list_counters[max_position]), rotation='vertical')
  plt.legend(get_datasets_names())
  plt.show()

In the following code, we load all the tags for considering all the datasets of train as a unique one and we count the number of occurrence for each of them.
After that, we plot their occurrences.

In [ ]:
# Count the occurrences of each tag and plot them
pos_freq_all_datasets = Counter([tag for sentence in get_pos_tags_file() for tag in sentence])
list_counters = [pos_freq_all_datasets]
plot_groupedBar_counters(list_counters)

Here we do the same as before, but for any training dataset to visualize the 
differences between the datasets.

In [ ]:
list_counters = []
# For each train dataset we take the save the occurrences of the tags
for dataset_name in datasets_train_dict_BIO.keys():
  tags = get_pos_tags_specific_file(dataset_name[:-4])
  pos_freq = Counter([tag for sentence in tags for tag in sentence])
  list_counters.append(pos_freq)

# Plot of all the counter in the same plot to compare them
plot_groupedBar_counters(list_counters)
plot_wordCloud_counters(list_counters)

## Indexing

Indexing is the process by which search engines organize information before a search to enable super-fast responses to queries. 
Searching through individual pages for keywords and topics would be a very slow process for search engines to identify relevant information. Instead, search engines use an inverted index, also known as a reverse index.
An inverted index is a system wherein a database of text elements is compiled along with pointers to the documents which contain those elements.



First of all, since we don't have a dataset of documents we decided to index the sentences inside our datasets.
To do that we have to create the sentences from the dictionary of datasets.

In [ ]:
sentences, labels, keys = get_all_sentences_from_datasets(datasets_train_dict_BIO)

Then, we create the dataset of sentences adding also the Id to indetify them while the query part.

In [ ]:
sentences_df = pd.DataFrame(columns=['docno', 'text'])

for i in range(len(sentences)):
    sentences_df.loc[i] = [f's{i}', ' '.join(sentences[i])]

In the following cell, we index the sentences dataframe. The index, with all its data structures, is written into a directory that we will call `index`.
`Index_Ref` provides the location where the index is stored.

In [ ]:
index_path = "./index"

indexer = pt.DFIndexer(index_path, overwrite=True)
index_ref = indexer.index(sentences_df['text'], sentences_df['docno'])
index_ref.toString()

Now we can now load the index and print it.
This is a Terrier index structure, which provides methods such as:
 - `getCollectionStatistics()`
 - `getInvertedIndex()`
 - `getLexicon()`

 Let's see what is returned by the `CollectionStatistics()` method.

In [ ]:
index = pt.IndexFactory.of(index_ref)
print(index.getCollectionStatistics().toString())

Now that we have indexed our documents, we can run a search over the document collection.
Here we used the TF-IDF weighting formula to rank the results. 

The `search()` method returns a dataframe with columns:
 - `qid`: the query identifier
 - `docid`: integer identifier for document 
 - `docno`: string identifier for document
 - `rank`: rank position
 - `score`: tf-idf score
 - `query`: the input query

In [ ]:
br = pt.BatchRetrieve(index, wmodel="TF_IDF")

queries = pd.DataFrame([["query1", "festa"], ["query2", "ammiraglio"], ["query3", "messaggio audio"]], columns=["qid", "query"])
br(queries)

## Conditional random fields

The conditional random fields model is a type of discriminative undirected probabilistic model. It can be seen as an extention of the Hidden Markov Models.

HMM take a sequence of data, in our case a sequence of words, and assumes that the observations are given by a set of states that we don't observe but which we are interested to tag. There are transition probabilities between the states in the sequence (probability of going into a specific next state) and there are emission probability for each state that states which is the probability of a observation in a given state.

HMM is a generative model: it aims to model the joint probability of hidden state and observation simultaniously. In this framework the aim is to understand the way the data we have are generated.

HMM has some limitation:
- transition and emission probabilities are static, in every position of the sentence in our case;
- each state depends only on the previous state but in natural language processing this is not the case, also the observations can depend on previous states in the speech.

Conditional random fields model can be imagined as an extension of the HMM where it is possible to define whatever dependency between the states and between the states and the observations. The graphical representation is an undirected graph where the nodes are the states and the observations. The states are connected with each other in an arbitrary way and a state depends on the state to which it is connected through an edge.

It is clear that it is a more general framework for catching the dependencies between the state and also between states and observations.

Conditional random fields want to model the probability of the state conditioned the observations. The model is not generative. In the case of named entity recognition the model has to discriminate whether a token is part of an entity or not, given the words of a sentence.
Often a variant of the method, called linear chain CRF, is used, in particular when dealing with big datasets. In case of LCCRF only the dependencies between consequent states are considered whereas all the relations between observations and hidden states are possible.

The underlying idea is similar to the one of the logistic regression.

$$\textbf{Mathematical model}$$

Consider a set generic function, in this setting called feature function $$f_{j}(\textbf{x}, y_{i-1}, y_{i}, i),$$
with $\textbf{x}$ observations of the problem, so in the NER case the words of the sentences, $y_{i-1}$ the i-1-th state in the hidden states of the CRF, so the previous state, $y_{i}$ the i-th state in the hidden states of the CRF, so subsequent to $y_{i-1}$, and $i$ is the index of the current state in the states chain, in the NER task it is the position in the sentence.

The $f_{j}$ functions can be any possible function defined over that inputs, obviously for obtaining good results in the sequence labeling task they should be meaningful to the application.

Given a set of feature functions, is then possible to define $F_j$ as
$$F_j(\textbf{y},\textbf{x}) = \sum_{i=1}^{N} f_j(\textbf{x}, y_{i-1}, y_{i}, i)$$
with $N$ length of the provided sequence of observations.

In this setting, we define $\theta_j$ as the weights of each function $F_j$, so $\theta_j$ is giving more or less importance to some functions with respect to others. These will be the variables to optimize in the optimization step.

The conditioned probability of the sequence of states $\textbf{y}$ given the observations $\textbf{x}$ and the parameters $\boldsymbol{\theta}$ is

$$p(\textbf{y}|\textbf{x}, \boldsymbol{\theta}) = \frac{1}{Z(\textbf{x})}e^{\boldsymbol{\theta}^T \cdot \textbf{F}(\textbf{y},\textbf{x})},$$

considering that $\textbf{F}(\textbf{y},\textbf{x})$ is the vectorial representation of the $F_j$ and $\boldsymbol{\theta}$ is the vectorial representation of the $\theta_j$ and $Z(\textbf{x})$ a normalizing factor used to obtain a probability distribution.

### Function definition

Here we define some function needed for performing the named entity recognition task using crf-suite.

In [ ]:
def standard_features_from_word(sentence, position, prefix='', punctuation_evaluation=False):
  '''
  Converts a word of the sentence into its representation given by a standard
  set of features that we consider the basis from which the feature vector can
  be enlarged

  This method has been implemented for having a crfsuite format representation
  of the dataset in order to apply use the conditional random fields framework
  provided by the sklearn_crfsuite package.

  Parameters
  ----------
  sentence: list
    list of the words composing the sentence.
  position: integer
    position in the list of the word that has to be converted into its feature 
    vector representation.
  prefix: str, optional
    string that can be appended before the names of the attributes of the 
    dictionary of features for specifying additional information.
  punctuation_evaluation: bool, optional
    True, to insert a feature about the fact that the word contains some
    punctiation elments, False to avoid inserting it.

  Returns
  -------
  standard_word_features: dict
    dictionary of the standard features of the word

  '''

  standard_word_features = None
  if position < len(sentence) and position >= 0:
    word = sentence[position]
    standard_word_features = {
          prefix+'word' : word,
          prefix+'word_lowercase' : word.lower(),
          prefix+'first_letter_upper' : word[0].isupper(),
          prefix+'all_letters_upper' : word.isupper(),
          prefix+'all_digits' : word.isdigit(),
          prefix+'position' : position,
    }
    if punctuation_evaluation:
      contains_punctuation = False
      for character in word:
        if character in string.punctuation:
          contains_punctuation = True
          break
      
      standard_word_features.update({prefix+'punctuation_inside' : contains_punctuation})

  return standard_word_features

################################################################################

def get_gazetteer_features(sentence, position, gazetteer_dict, gazetteer_entity_lenght=3,
                           prefix='', bio_tags=True):
  '''
  Returns the features related to the belonging of a word of the sentence to the 
  list of known entities.

  This method has been implemented for having a crfsuite format representation
  of the dataset in order to apply use the conditional random fields framework
  provided by the sklearn_crfsuite package.

  Parameters
  ----------
  sentence: list
    list of the words composing the sentence.
  position: integer
    position in the list of the word that has to be converted into its feature 
    vector representation.
  gazetteer_dict: dict
    dictionary containing a elements the sets of the known entities, each set 
    contains a different type of entity.
  gazetteer_entity_lenght: integer
    maximum length, in terms of number of words, of the known named entities.
    Incresing the length of the names of the entities will end up in a bigger 
    computational complexity but could end up in better results.
  prefix: str, optional
    string that can be appended before the names of the attributes of the 
    dictionary of features for specifying additional information.
  bio_tags: bool, optional
    True, for having a sort of begin-inside-outside notation in features related
    to the known entities, False, for having only the inside-outside notation.

  Returns
  -------
  gazetteer_features: dict
    dictionary of the features of the word reguarding the belonging of the word
    to the know entity lists.
    
  '''

  gazetteer_features = {}
  for key in gazetteer_dict:
    found = False
    for ent_len in range(1, gazetteer_entity_lenght+1):
      for i in range(ent_len):
        if position - i >= 0 and position + ent_len - i - 1 < len(sentence):
          entity = ' '.join(sentence[(position - i):(position + ent_len - i - 1 + 1)])
          if i == 0:
            if entity in gazetteer_dict[key]:
              if bio_tags:
                gazetteer_features.update({prefix+'B-'+key: True})
              else:
                gazetteer_features.update({prefix+key: True})
              found = True
          else:
            if entity in gazetteer_dict[key]:
              if bio_tags:
                gazetteer_features.update({prefix+'I-'+key: True})
              else:
                gazetteer_features.update({prefix+key: True})
              found = True
    if not found:
      gazetteer_features.update({key: False})

  return gazetteer_features
    
################################################################################

def transform_word_to_features(sentence, position, postags=None, embeddings=None,
                               previous_words_to_embed=1, next_words_to_embed=1,
                               gazetteer=None, gazetteer_entity_lenght=3, bio_tags=True,
                               punctuation_evaluation=False):
  '''
  Converts a word of the sentence into its representation given by a set of
  features. Some features are inserted by default some others can be requested
  through setting the optional parameters.

  This method has been implemented for having a crfsuite format representation
  of the dataset in order to apply use the conditional random fields framework
  provided by the sklearn_crfsuite package.

  Parameters
  ----------
  sentence: list
    list of the words composing the sentence.
  position: integer
    position in the list of the word that has to be converted into its feature 
    vector representation.
  postags: list, optional
    list of the part-of-speech tags related to the words of the sentence. This
    list has to be of the same length of the sentence list.
  embeddings: list, optional
    list of the embeddings related to the words of the sentence. This list has 
    to be of the same length of the sentence list.
  previous_words_to_embed: integer, optional
    number of words preceeding the considered word that hava to be embedded in 
    the feature representation of the considered word.
  next_words_to_embed: integer, optional
    number of words following the considered word that hava to be embedded in 
    the feature representation of the considered word.
  gazetteer: dict, optional
    dictionaty containing the known named entities. If None, it is assumed that
    gazetteer cannot be used.
  gazetteer_entity_lenght: integer, optional
    maximum length, in terms of number of words, of the known named entities.
    Incresing the length of the names of the entities will end up in a bigger 
    computational complexity but could end up in better results.
  bio_tags: bool, optional
    True, for having a sort of begin-inside-outside notation in features related
    to the known entities, False, for having only the inside-outside notation.
  punctuation_evaluation: bool, optional
    True, to insert a feature about the fact that the word contains some
    punctiation elments, False to avoid inserting it.

  Returns
  -------
  word_features: dict
    dictionary of the features of the word

  '''

  word_features = None
  if position < len(sentence) and position >= 0:
    # Getting the standard features for the word to convert into features
    word_features = standard_features_from_word(sentence, position, punctuation_evaluation=punctuation_evaluation)

    # Adding the embedding representation of the word to the features
    if embeddings != None:
      embedding = embeddings[position]
      length = len(embedding)
      for i in range(length):
        word_features.update({'dim'+str(i)+'-embedding' : embedding[i]})

    # Adding the POS tag to the features of the word
    if postags != None:
      word_features.update({'postag': postags[position]})
    
    # Embedding the context of the sentence inside the feature representation of
    # the word, adding the features of the previous words in the sentence
    if previous_words_to_embed > 0:
      i = 1
      while i < previous_words_to_embed + 1 and position-i >= 0:
        word_features.update(standard_features_from_word(sentence, position-i, prefix='-'+str(i)+':', punctuation_evaluation=punctuation_evaluation))
        if postags != None:
          word_features.update({'-'+str(i)+':postag': postags[position-i]})
        i += 1

    # Embedding the context of the sentence inside the feature representation of
    # the word, adding the features of the previous words in the sentence
    if next_words_to_embed > 0:
      i = 1
      while i < next_words_to_embed + 1 and position+i <= len(sentence)-1:
        word_features.update(standard_features_from_word(sentence, position+i, prefix='+'+str(i)+':', punctuation_evaluation=punctuation_evaluation))
        if postags != None:
          word_features.update({'+'+str(i)+':postag': postags[position+i]})
        i += 1

    if gazetteer != None:
      word_features.update(get_gazetteer_features(sentence, position, gazetteer, bio_tags=bio_tags))
  else:
    print('Wrong input, the index of the word in the sentence is not valid.')
  return word_features 

################################################################################

def transform_sentences_to_features(sentences, postags=None, embeddings=None, 
                                    previous_words_to_embed=1, next_words_to_embed=1,
                                    gazetteer=None, gazetteer_entity_lenght=3, bio_tags=True,
                                    punctuation_evaluation=False):
  '''
  Converts the sentences into their representation given by a list of 
  dictionaries of features. Some features are inserted by default some others 
  can be requested through setting the optional parameters.

  This method has been implemented for having a crfsuite format representation
  of the dataset in order to apply use the conditional random fields framework
  provided by the sklearn_crfsuite package.

  Parameters
  ----------
  sentence: list
    list of lists of the words composing the sentences.
  postag: list, optional
    list of lists of the part-of-speech tags related to the words of the 
    sentences. This list of lists has to have the same dimensions of the 
    sentence list.
  embeddings: list, optional
    list of lists of the embeddings related to the words of the sentences. This 
    list of lists has to have the same dimensions of the sentence list.
  previous_words_to_embed: integer, optional
    number of words preceeding the considered word that have to be embedded in 
    the feature representation of the considered word.
  next_words_to_embed: integer, optional
    number of words following the considered word that havae to be embedded in 
    the feature representation of the considered word.
  gazetteer: dict, optional
    dictionaty containing the known named entities. If None, it is assumed that
    gazetteer cannot be used.
  gazetteer_entity_lenght: integer, optional
    maximum length, in terms of number of words, of the known named entities.
    Incresing the length of the names of the entities will end up in a bigger 
    computational complexity but could end up in better results.
  bio_tags: bool, optional
    True, for having a sort of begin-inside-outside notation in features related
    to the known entities, False, for having only the inside-outside notation.
  punctuation_evaluation: bool, optional
    True, to insert a feature about the fact that the word contains some
    punctiation elments, False to avoid inserting it.

  Returns
  -------
  sentence_features: list
    representation of the sentences as a list of lists of dictionaries of features
  
  '''
  
  all_sentences_features = []
  for index,sentence in tqdm(enumerate(sentences)):
    sentence_features = []
    for i in range(len(sentence)):
      sentence_features.append(transform_word_to_features(sentence, i, postags=(postags[index] if postags != None else None), 
            embeddings=(embeddings[index] if embeddings != None else None), previous_words_to_embed=previous_words_to_embed, 
            next_words_to_embed=next_words_to_embed, gazetteer=gazetteer,
            gazetteer_entity_lenght=gazetteer_entity_lenght, bio_tags=bio_tags,
            punctuation_evaluation=punctuation_evaluation))
    all_sentences_features.append(sentence_features)
  return all_sentences_features


In [ ]:
# Providing example of the format of input that takes the functions generating
# the features from the datasets
sentences_input_example = [
    ['Mark', 'Carman', 'è', 'fantastico!'],
    ['Anche', 'il', 'dottor', 'Scotti', 'è', 'fantastico!'], 
    ['Ciao', 'sono', 'Mike', 'e', 'sono', 'a', 'Milano']]

# Providing example of the output given by the functions generating the features
# from the datasets
output_features_example = transform_sentences_to_features(sentences_input_example, punctuation_evaluation=True)
output_features_example

### Application of the CRF to the dataset

In [ ]:
# Getting the sentences and the labels from the training dataset
sentences_merged_train_BIO, labels_merged_train_BIO, _ = get_all_sentences_from_datasets(datasets_train_dict_BIO, lower=False)
# Getting the sentences and the labels from the test dataset
sentences_merged_test_BIO, labels_merged_test_BIO, _ = get_all_sentences_from_datasets(datasets_test_dict_BIO, lower=False)

In [ ]:
# Definition of a gazetteer set
PATH_TO_GAZETTEER = '/content/KIND_project/datasets/Entities'
gazetteer_loc = set(line.replace('LOC','').strip() for line in open(PATH_TO_GAZETTEER + '/it-LOC-wikipedia.txt', encoding='utf-8'))
gazetteer_org = set(line.replace('ORG','').strip() for line in open(PATH_TO_GAZETTEER + '/it-ORG-wikipedia.txt', encoding='utf-8'))
gazetteer_per = set(line.replace('PER','').strip() for line in open(PATH_TO_GAZETTEER + '/it-PER-wikipedia.txt', encoding='utf-8'))
gazetteer_dict = {
    'LOC': gazetteer_loc,
    'ORG': gazetteer_org,
    'PER': gazetteer_per,
}
gazetteer = gazetteer_loc.union(gazetteer_org).union(gazetteer_per)

#### Fitting the model without POS tags and without gazetteer

In [ ]:
# Converting the training dataset in the one required for training the model to
# perform the named entity recognition task through the conditional random 
# fields method
sentences_features_merged_train_BIO = transform_sentences_to_features(sentences_merged_train_BIO, 
                                                                      previous_words_to_embed=1,
                                                                      next_words_to_embed=1)

# Converting the test dataset in the one required for performing the named 
# entity recognition task through the conditional random fields method
sentences_features_merged_test_BIO = transform_sentences_to_features(sentences_merged_test_BIO, 
                                                                     previous_words_to_embed=1,
                                                                     next_words_to_embed=1)

In [ ]:
#Creating the CRF model for NER
ner_crf_model = CRF(algorithm='lbfgs',
                    c1=0.1,
                    c2=0.1,
                    max_iterations=100,
                    verbose=True,
                    all_possible_transitions=False)

# Fitting the CRF model using the training set
ner_crf_model.fit(sentences_features_merged_train_BIO, labels_merged_train_BIO)

In [ ]:
# Defining the labels on which the performance evaluation has to be done
classes_BIO = list(ner_crf_model.classes_)
# Removing the class of words that are not entities in order to have the 
# evaluation of the performance considering only the classes of the entities
classes_BIO.remove('O')

In [ ]:
# Predicting the lables on the train set
prediction_merged_train_BIO = ner_crf_model.predict(sentences_features_merged_train_BIO)
# Predicting the lables on the test set
prediction_merged_test_BIO = ner_crf_model.predict(sentences_features_merged_test_BIO)

# Computing the flat F1 score on the train set
f1_score_train_BIO = crf_metrics.flat_f1_score(labels_merged_train_BIO, prediction_merged_train_BIO,
                                       average='weighted', labels=classes_BIO)
# Computing the flat F1 score on the test set
f1_score_test_BIO = crf_metrics.flat_f1_score(labels_merged_test_BIO, prediction_merged_test_BIO,
                                      average='weighted', labels=classes_BIO)

print('Flat F1 score on the trainset: ' + str(f1_score_train_BIO))
print('Flat F1 score on the testset: ' + str(f1_score_test_BIO))

In [ ]:
# Computing the main performance metrics on the test set
classification_report_test_IO = crf_metrics.flat_classification_report(labels_merged_test_BIO, prediction_merged_test_BIO,
                                      labels=classes_BIO)
print('Classification report on the testset: ')
print(classification_report_test_IO)

In [ ]:
print('Performance on the trainset')
performance_merged_train_BIO = performance_metrics(prediction_merged_train_BIO, labels_merged_train_BIO)
performance_merged_train_BIO

In [ ]:
print('Performance on the testset')
performance_merged_test_BIO = performance_metrics(prediction_merged_test_BIO, labels_merged_test_BIO)
performance_merged_test_BIO

In [ ]:
eli5.explain_weights(ner_crf_model, top=100)

The model is learning mainly the words to recognise as entities and it is learning also the ones that are outside, in particular we can identify a lot of stopwords in the outside class.

#### Fitting the model with also gazetteer

In [ ]:
embedding_train = [[get_word2vec_vector(sentences_merged_train_BIO[i][j], embeddings_model_train) for j in range(len(sentences_merged_train_BIO[i]))] for i in range(len(sentences_merged_train_BIO))]
embedding_test = [[get_word2vec_vector(sentences_merged_test_BIO[i][j], embeddings_model_train) for j in range(len(sentences_merged_test_BIO[i]))] for i in range(len(sentences_merged_test_BIO))]

In [ ]:
# Converting the training dataset in the one required for training the model to
# perform the named entity recognition task through the conditional random 
# fields method, here it is considered also whether a word, or sequence of words,
# is in the set of known entities.
sentences_features_merged_train_BIO_gaz = transform_sentences_to_features(sentences_merged_train_BIO,
                                                                          previous_words_to_embed=1, 
                                                                          next_words_to_embed=1,
                                                                          embeddings=embedding_train,
                                                                          gazetteer = gazetteer_dict,
                                                                          gazetteer_entity_lenght=6)

# Converting the test dataset in the one required for test the model to perform
# the named entity recognition task through the conditional random fields method,
# here it is considered also whether a word, or sequence of words, is in the set
# of known entities.
sentences_features_merged_test_BIO_gaz = transform_sentences_to_features(sentences_merged_test_BIO,
                                                                         previous_words_to_embed=1,
                                                                         next_words_to_embed=1,
                                                                         embeddings=embedding_test,
                                                                         gazetteer = gazetteer_dict,
                                                                         gazetteer_entity_lenght=6)

In [ ]:
#Creating the CRF model for NER
ner_crf_model_gaz = CRF(algorithm='lbfgs',
                        c1=0.1,
                        c2=0.1,
                        max_iterations=80,
                        verbose=True,
                        all_possible_transitions=False)

# Fitting the CRF model using the training set
ner_crf_model_gaz.fit(sentences_features_merged_train_BIO_gaz, labels_merged_train_BIO)

In [ ]:
# Defining the labels on which the performance evaluation has to be done
classes_BIO_gaz = list(ner_crf_model_gaz.classes_)
# Removing the class of words that are not entities in order to have the 
# evaluation of the performance considering only the classes of the entities
classes_BIO_gaz.remove('O')

In [ ]:
# Predicting the lables on the train set
prediction_merged_train_BIO_gaz = ner_crf_model_gaz.predict(sentences_features_merged_train_BIO_gaz)
# Predicting the lables on the test set
prediction_merged_test_BIO_gaz = ner_crf_model_gaz.predict(sentences_features_merged_test_BIO_gaz)

# Computing the flat F1 score on the train set
f1_score_train_BIO_gaz = crf_metrics.flat_f1_score(labels_merged_train_BIO, prediction_merged_train_BIO_gaz,
                                       average='weighted', labels=classes_BIO_gaz)
# Computing the flat F1 score on the test set
f1_score_test_BIO_gaz = crf_metrics.flat_f1_score(labels_merged_test_BIO, prediction_merged_test_BIO_gaz,
                                      average='weighted', labels=classes_BIO_gaz)

print('Flat F1 score on the trainset: ' + str(f1_score_train_BIO_gaz))
print('Flat F1 score on the testset: ' + str(f1_score_test_BIO_gaz))

In [ ]:
# Computing the main performance metrics on the test set
classification_report_test_IO_gaz = crf_metrics.flat_classification_report(labels_merged_test_BIO, prediction_merged_test_BIO_gaz,
                                      labels=classes_BIO_gaz)
print('Classification report on the testset: ')
print(classification_report_test_IO_gaz)

In [ ]:
print('Performance on the trainset')
performance_merged_train_BIO_gaz = performance_metrics(prediction_merged_train_BIO_gaz, labels_merged_train_BIO)
performance_merged_train_BIO_gaz

In [ ]:
print('Performance on the testset')
performance_merged_test_BIO_gaz = performance_metrics(prediction_merged_test_BIO_gaz, labels_merged_test_BIO)
performance_merged_test_BIO_gaz

In [ ]:
eli5.explain_weights(ner_crf_model_gaz, top=100)

#### Fitting the model with also POS tags

In [ ]:
# Converting the training dataset in the one required for training the model to
# perform the named entity recognition task through the conditional random 
# fields method, here it is considered also whether a word, or sequence of words,
# is in the set of known entities. The features in this section include also the
# part-of-speech tags
sentences_features_merged_train_BIO_gaz_pos = transform_sentences_to_features(sentences_merged_train_BIO,
                                                                          postags = get_pos_tags(sentences_merged_train_BIO),
                                                                          previous_words_to_embed=2, 
                                                                          next_words_to_embed=2,
                                                                          gazetteer = gazetteer_dict,
                                                                          gazetteer_entity_lenght = 6)

# Converting the test dataset in the one required for test the model to perform
# the named entity recognition task through the conditional random fields method,
# here it is considered also whether a word, or sequence of words, is in the set
# of known entities. The features in this section include also the 
# part-of-speech tags
sentences_features_merged_test_BIO_gaz_pos = transform_sentences_to_features(sentences_merged_test_BIO,
                                                                             postags = get_pos_tags(sentences_merged_test_BIO),
                                                                             previous_words_to_embed=2,
                                                                             next_words_to_embed=2,
                                                                             gazetteer = gazetteer_dict,
                                                                             gazetteer_entity_lenght = 6)

In [ ]:
#Creating the CRF model for NER
ner_crf_model_gaz_pos = CRF(algorithm='lbfgs',
                        c1=0.1,
                        c2=0.1,
                        max_iterations=150,
                        verbose=True,
                        all_possible_transitions=False)

# Fitting the CRF model using the training set
ner_crf_model_gaz_pos.fit(sentences_features_merged_train_BIO_gaz_pos, labels_merged_train_BIO)

In [ ]:
# Defining the labels on which the performance evaluation has to be done
classes_BIO_gaz_pos = list(ner_crf_model_gaz_pos.classes_)
# Removing the class of words that are not entities in order to have the 
# evaluation of the performance considering only the classes of the entities
classes_BIO_gaz_pos.remove('O')

In [ ]:
# Predicting the lables on the train set
prediction_merged_train_BIO_gaz_pos = ner_crf_model_gaz_pos.predict(sentences_features_merged_train_BIO_gaz_pos)
# Predicting the lables on the test set
prediction_merged_test_BIO_gaz_pos = ner_crf_model_gaz_pos.predict(sentences_features_merged_test_BIO_gaz_pos)

# Computing the flat F1 score on the train set
f1_score_train_BIO_gaz_pos = crf_metrics.flat_f1_score(labels_merged_train_BIO, prediction_merged_train_BIO_gaz_pos,
                                       average='weighted', labels=classes_BIO_gaz_pos)
# Computing the flat F1 score on the test set
f1_score_test_BIO_gaz_pos = crf_metrics.flat_f1_score(labels_merged_test_BIO, prediction_merged_test_BIO_gaz_pos,
                                      average='weighted', labels=classes_BIO_gaz_pos)

print('Flat F1 score on the trainset: ' + str(f1_score_train_BIO_gaz_pos))
print('Flat F1 score on the testset: ' + str(f1_score_test_BIO_gaz_pos))

In [ ]:
# Computing the main performance metrics on the test set
classification_report_test_IO_gaz_pos = crf_metrics.flat_classification_report(labels_merged_test_BIO, prediction_merged_test_BIO_gaz_pos,
                                      labels=classes_BIO_gaz_pos)
print('Classification report on the testset: ')
print(classification_report_test_IO_gaz_pos)

In [ ]:
print('Performance on the trainset')
performance_merged_train_BIO_gaz_pos = performance_metrics(prediction_merged_train_BIO_gaz_pos, labels_merged_train_BIO)
performance_merged_train_BIO_gaz_pos

In [ ]:
print('Performance on the testset')
performance_merged_test_BIO_gaz_pos = performance_metrics(prediction_merged_test_BIO_gaz_pos, labels_merged_test_BIO)
performance_merged_test_BIO_gaz_pos

In [ ]:
eli5.explain_weights(ner_crf_model_gaz_pos, top=100)

#### Fitting the model with also embeddings representation of the words

In [ ]:
embedding_train = [[get_word2vec_vector(sentences_merged_train_BIO[i][j], embeddings_model_train) for j in range(len(sentences_merged_train_BIO[i]))] for i in range(len(sentences_merged_train_BIO))]
embedding_test = [[get_word2vec_vector(sentences_merged_test_BIO[i][j], embeddings_model_train) for j in range(len(sentences_merged_test_BIO[i]))] for i in range(len(sentences_merged_test_BIO))]

In [ ]:
embedding_train[0][1]

In [ ]:
# Converting the training dataset in the one required for training the model to
# perform the named entity recognition task through the conditional random 
# fields method with as features also the embedding representation of the words
sentences_features_merged_train_BIO_emb = transform_sentences_to_features(sentences_merged_train_BIO,
                                                                          embeddings = embedding_train)

# Converting the test dataset in the one required for performing the named 
# entity recognition task through the conditional random fields method with as 
# features also the embedding representation of the words
sentences_features_merged_test_BIO_emb = transform_sentences_to_features(sentences_merged_test_BIO,
                                                                         embeddings = embedding_test)

In [ ]:
#Creating the CRF model for NER
ner_crf_model_emb = CRF(algorithm='lbfgs',
                    c1=0.1,
                    c2=0.1,
                    max_iterations=80,
                    verbose=True,
                    all_possible_transitions=False)

# Fitting the CRF model using the training set
ner_crf_model_emb.fit(sentences_features_merged_train_BIO_emb, labels_merged_train_BIO)

In [ ]:
# Defining the labels on which the performance evaluation has to be done
classes_BIO_emb = list(ner_crf_model_emb.classes_)
# Removing the class of words that are not entities in order to have the 
# evaluation of the performance considering only the classes of the entities
classes_BIO_emb.remove('O')

In [ ]:
# Predicting the lables on the train set
prediction_merged_train_BIO_emb = ner_crf_model_emb.predict(sentences_features_merged_train_BIO_emb)
# Predicting the lables on the test set
prediction_merged_test_BIO_emb = ner_crf_model_emb.predict(sentences_features_merged_test_BIO_emb)

# Computing the flat F1 score on the train set
f1_score_train_BIO_emb = crf_metrics.flat_f1_score(labels_merged_train_BIO, prediction_merged_train_BIO_emb,
                                       average='weighted', labels=classes_BIO_emb)
# Computing the flat F1 score on the test set
f1_score_test_BIO_emb = crf_metrics.flat_f1_score(labels_merged_test_BIO, prediction_merged_test_BIO_emb,
                                      average='weighted', labels=classes_BIO_emb)

print('Flat F1 score on the trainset: ' + str(f1_score_train_BIO_emb))
print('Flat F1 score on the testset: ' + str(f1_score_test_BIO_emb))

In [ ]:
# Computing the main performance metrics on the test set
classification_report_test_IO_emb = crf_metrics.flat_classification_report(labels_merged_test_BIO, prediction_merged_test_BIO_emb,
                                      labels=classes_BIO_emb)
print('Classification report on the testset: ')
print(classification_report_test_IO_emb)

In [ ]:
print('Performance on the trainset')
performance_merged_train_BIO_emb = performance_metrics(prediction_merged_train_BIO_emb, labels_merged_train_BIO)
performance_merged_train_BIO_emb

In [ ]:
print('Performance on the testset')
performance_merged_test_BIO_emb = performance_metrics(prediction_merged_test_BIO_emb, labels_merged_test_BIO)
performance_merged_test_BIO_emb

In [ ]:
eli5.explain_weights(ner_crf_model_emb, top=100)

#### Fitting the model using the data with only the inside-outside notation

In [ ]:
# Getting the sentences and the labels from the training dataset
sentences_merged_train_IO, labels_merged_train_IO, _ = get_all_sentences_from_datasets(datasets_train_dict_IO, lower=False)
# Getting the sentences and the labels from the test dataset
sentences_merged_test_IO, labels_merged_test_IO, _ = get_all_sentences_from_datasets(datasets_test_dict_IO, lower=False)

In [ ]:
# Converting the training dataset without BIO notation in the format required
# for training the model to perform the named entity recognition task through 
# the conditional random fields method, here it is considered also whether a word,
# or sequence of words, is in the set of known entities. The features in this 
# section include also the part-of-speech tags.
sentences_features_merged_train_IO_gaz_pos = transform_sentences_to_features(sentences_merged_train_IO,
                                                                          postags = get_pos_tags_file(),
                                                                          previous_words_to_embed=2, 
                                                                          next_words_to_embed=2,
                                                                          gazetteer = gazetteer_dict,
                                                                          gazetteer_entity_lenght=6,
                                                                          bio_tags=False)

# Converting the test dataset without BIO notation in the format required for
# test the model to perform the named entity recognition task through the 
# conditional random fields method, here it is considered also whether a word,
# or sequence of words, is in the set of known entities. The features in this 
# section include also the part-of-speech tags.
sentences_features_merged_test_IO_gaz_pos = transform_sentences_to_features(sentences_merged_test_IO,
                                                                            postags = get_pos_tags_file(train_or_test=False),
                                                                            previous_words_to_embed=2,
                                                                            next_words_to_embed=2,
                                                                            gazetteer = gazetteer_dict,
                                                                            gazetteer_entity_lenght=6,
                                                                            bio_tags=False)

In [ ]:
#Creating the CRF model for NER
ner_crf_model_IO_gaz_pos = CRF(algorithm='lbfgs',
                        c1=0.1,
                        c2=0.1,
                        max_iterations=100,
                        verbose=True,
                        all_possible_transitions=False)

# Fitting the CRF model using the training set
ner_crf_model_IO_gaz_pos.fit(sentences_features_merged_train_IO_gaz_pos, labels_merged_train_IO)

In [ ]:
# Defining the labels on which the performance evaluation has to be done
classes_IO_gaz_pos = list(ner_crf_model_IO_gaz_pos.classes_)
# Removing the class of words that are not entities in order to have the 
# evaluation of the performance considering only the classes of the entities
classes_IO_gaz_pos.remove('O')

In [ ]:
# Predicting the lables on the train set
prediction_merged_train_IO_gaz_pos = ner_crf_model_IO_gaz_pos.predict(sentences_features_merged_train_IO_gaz_pos)
# Predicting the lables on the test set
prediction_merged_test_IO_gaz_pos = ner_crf_model_IO_gaz_pos.predict(sentences_features_merged_test_IO_gaz_pos)

# Computing the flat F1 score on the train set
f1_score_train_IO_gaz_pos = crf_metrics.flat_f1_score(labels_merged_train_IO, prediction_merged_train_IO_gaz_pos,
                                       average='weighted', labels=classes_IO_gaz_pos)
# Computing the flat F1 score on the test set
f1_score_test_IO_gaz_pos = crf_metrics.flat_f1_score(labels_merged_test_IO, prediction_merged_test_IO_gaz_pos,
                                      average='weighted', labels=classes_IO_gaz_pos)

print('Flat F1 score on the trainset: ' + str(f1_score_train_IO_gaz_pos))
print('Flat F1 score on the testset: ' + str(f1_score_test_IO_gaz_pos))

In [ ]:
# Computing the flat F1 score on the test set
f1_score_test_IO_gaz_pos = crf_metrics.flat_classification_report(labels_merged_test_IO, prediction_merged_test_IO_gaz_pos,
                                      average='weighted', labels=classes_IO_gaz_pos)
print('Flat F1 score on the trainset: ' + str(f1_score_test_IO_gaz_pos))

In [ ]:
print('Performance on the trainset')
performance_merged_train_IO_gaz_pos = performance_metrics(prediction_merged_train_IO_gaz_pos, labels_merged_train_IO)
performance_merged_train_IO_gaz_pos

In [ ]:
print('Performance on the testset')
performance_merged_test_IO_gaz_pos = performance_metrics(prediction_merged_test_IO_gaz_pos, labels_merged_test_IO)
performance_merged_test_IO_gaz_pos

In [ ]:
eli5.explain_weights(ner_crf_model_IO_gaz_pos, top=100)

#### Performance

In [ ]:
from itertools import combinations

dataset_names_train_BIO = ['degasperi_train_BIO.tsv', 'fiction_train_BIO.tsv','moro_train_BIO.tsv', 'wikinews_train_BIO.tsv']
dataset_names_test_BIO = ['degasperi_test_BIO.tsv', 'fiction_test_BIO.tsv','moro_test_BIO.tsv', 'wikinews_test_BIO.tsv']

def generate_unordered_tuples(s, limit=10000000):
    # Convert set to list to ensure consistent ordering
    elements = list(s)
    n = len(elements)

    # Generate tuples of different lengths from 0 to n
    tuples = []
    for r in range(1, min(n + 1, limit)):
        tuples.extend(combinations(elements, r))

    
    return tuples

#################################################################################################################

def extract_sentence_labels(datasets_train_dict_BIO, datasets_test_dict_BIO):
    # Getting the sentences and the labels from the training dataset
    sentences_train, labels_train, _ = get_all_sentences_from_datasets(datasets_train_dict_BIO, lower=False)
    # Getting the sentences and the labels from the test dataset
    sentences_test, labels_test, _ = get_all_sentences_from_datasets(datasets_test_dict_BIO, lower=False)

    return sentences_train, labels_train, sentences_test, labels_test

#################################################################################################################

# Definition of a gazetteer set
PATH_TO_GAZETTEER = '/content/KIND_project/datasets/Entities'
gazetteer_loc = set(line.replace('LOC','').strip() for line in open(PATH_TO_GAZETTEER + '/it-LOC-wikipedia.txt', encoding='utf-8'))
gazetteer_org = set(line.replace('ORG','').strip() for line in open(PATH_TO_GAZETTEER + '/it-ORG-wikipedia.txt', encoding='utf-8'))
gazetteer_per = set(line.replace('PER','').strip() for line in open(PATH_TO_GAZETTEER + '/it-PER-wikipedia.txt', encoding='utf-8'))
gazetteer_dict = {
    'LOC': gazetteer_loc,
    'ORG': gazetteer_org,
    'PER': gazetteer_per,
}
gazetteer = gazetteer_loc.union(gazetteer_org).union(gazetteer_per)

#################################################################################################################

# le altre abbiamo detto che non servono

###################################################################################################################
##soloquesta

def get_CRF_POSandGazetters_model(sentences_train, labels_train, sentences_test): 
    # Converting the training dataset in the one required for training the model to
    # perform the named entity recognition task through the conditional random 
    # fields method
    sentences_features_train_gaz_pos = transform_sentences_to_features(sentences_train, postags = get_pos_tags(sentences_train), gazetteer = gazetteer_dict)

    # Converting the test dataset in the one required for performing the named 
    # entity recognition task through the conditional random fields method
    sentences_features_test_gaz_pos = transform_sentences_to_features(sentences_test, postags = get_pos_tags(sentences_test), gazetteer = gazetteer_dict)

    sentences_features_train_gaz_pos[0]

    #Creating the CRF model for NER
    ner_crf_model_gaz_pos = CRF(algorithm='lbfgs',
                            c1=0.5,
                            c2=0.5,
                            max_iterations=100,
                            verbose=True,
                            all_possible_transitions=False)

    # Fitting the CRF model using the training set
    ner_crf_model_gaz_pos.fit(sentences_features_train_gaz_pos, labels_train)

    return ner_crf_model_gaz_pos, sentences_features_train_gaz_pos, sentences_features_test_gaz_pos

def predict_and_evaluate(sentences_features_train_gaz_pos, sentences_features_test_gaz_pos, labels_train, labels_test, ner_crf_model_gaz_pos):
    # Predicting the lables on the train set
    prediction_train_BIO = ner_crf_model_gaz_pos.predict(sentences_features_train_gaz_pos)

    # Predicting the lables on the test set
    prediction_test_BIO = ner_crf_model_gaz_pos.predict(sentences_features_test_gaz_pos)

    ###
    # in qualche modo qui calcolo le performance varie e le salvo da qualche parte
    ###

    eli5.show_weights(ner_crf_model_gaz_pos, top=30)

    performance_train = performance_metrics(prediction_train_BIO, labels_train)

    performance_test = performance_metrics(prediction_test_BIO, labels_test)

    return performance_train, performance_test

#################################################################################################################

def performance_metrics(predicted_labels, real_labels):
  '''
  Computes performance metrics given the predictions and the true labels

  Parameters
  ----------
  predictions: ndarray
    predictions of samples obtained with a model
  true_labels: ndarray
    true labels of the samples

  Returns
  -------
  metrics_df: DataFrame
    DataFrame containing the statistics contained in the parameter metrics_df
    plus the statistics computed on the new predictions

  '''
  
  metrics_df = None
  classes = set(element for sentence in real_labels for element in sentence)
  for tag in classes:
    new_predicted_labels = np.array([1 if element == tag else 0 for sentence in predicted_labels for element in sentence])
    new_real_labels = np.array([1 if element == tag else 0 for sentence in real_labels for element in sentence])
    metrics_df = metrics(new_predicted_labels, new_real_labels, metrics_df=metrics_df, class_label=tag)
  
  return metrics_df

#################################################################################################################

  

subsets_dataset_names_train_BIO = generate_unordered_tuples(set(dataset_names_train_BIO))
subsets_dataset_names_test_BIO = generate_unordered_tuples(set(dataset_names_test_BIO))


print(subsets_dataset_names_train_BIO)
print(subsets_dataset_names_test_BIO)


subsets_datasets_train_dict_BIO = []
subsets_datasets_test_dict_BIO = []

for i in range(0, len(subsets_dataset_names_train_BIO)):
    current_subset_train_dict = {}
    current_subset_test_dict = {}
    for j in range(0, len(subsets_dataset_names_train_BIO[i])):
      current_subset_train_dict[subsets_dataset_names_train_BIO[i][j]] = datasets_train_dict_BIO[subsets_dataset_names_train_BIO[i][j]]
      current_subset_test_dict[subsets_dataset_names_test_BIO[i][j]] = datasets_test_dict_BIO[subsets_dataset_names_test_BIO[i][j]]
    #current_subset_test_dict['dataset_name'] = subsets_dataset_names_test_BIO[i]
    #current_subset_train_dict['dataset_name'] = subsets_dataset_names_train_BIO[i]
    subsets_datasets_train_dict_BIO.append(current_subset_train_dict)
    subsets_datasets_test_dict_BIO.append(current_subset_test_dict)


print(len(subsets_datasets_train_dict_BIO))

results = []

for i in range(0, len(subsets_datasets_train_dict_BIO)):
    print('#'*30 + ' ' + str(i) + ' ' + '#'*30)
    sentences_train, labels_train, sentences_test, labels_test = extract_sentence_labels(subsets_datasets_train_dict_BIO[i], subsets_datasets_test_dict_BIO[i])

    ner_crf_model_gaz_pos, sentences_features_train_gaz_pos, sentences_features_test_gaz_pos = get_CRF_POSandGazetters_model(sentences_train, labels_train, sentences_test)

    res_train, res_test = predict_and_evaluate(sentences_features_train_gaz_pos, 
                                                    sentences_features_test_gaz_pos, 
                                                    labels_train, 
                                                    labels_test, 
                                                    ner_crf_model_gaz_pos)
                                                    #subsets_datasets_train_dict_BIO[i]['dataset_name'].replace('_train.tsv', '')))
    results.append({'dataset_name': str(subsets_dataset_names_train_BIO[i]).replace('_train_BIO.tsv', '').replace('(', '').replace(')', '').replace(',', '+').replace("'", ""),
                    'results_train': res_train,
                    'results_test': res_test
                    })
import pickle    
with open('data.pkl', 'wb') as file:
    pickle.dump(results, file)

with open('data.pkl', 'rb') as file:
    loaded_variable = pickle.load(file)

print(loaded_variable)


## NER Transformers

A transformer is a deep learning model that is different from the classical models since it adopts **self-attention.** It is used primarly in the fields of NLP and computer vision.
In general there are two big families of transformers:
- Transformer encoder layers, like BERT.
- Transformer decoder layers, like GPT.

Bert was pre-trained for two tasks:
- Language modeling;
- Next sentence prediction

As a result of this training process, BERT learns latent representations of words and sentences in context. After pre-training, BERT can be **fine-tuned** with fewer resources on smaller datasets to optimize its performance on specific tasks such as NLP tasks (language inference, text classification) and sequence-to-sequence based language generation tasks (question-answering, conversational response generation).

In our case we will fine-tune BERT for Token Classification, that takes as input a sequence of tokens and the respective entity tag.

Since the dataset is already tokenized and cleaned by the authors we won't apply any type of pre-processing.

We will use two different models pre-trained for the italian language:
- bert italian xxl cased
- bert base multilingual.

For each of them we performed cross-validation to choose the most important hyperparameters.



In [ ]:
def createDataset(train, test):

  '''
  Takes two dataframes and it creates an Hugging Face Dictionary of Datasets

  Parameters
  ----------
  train: DataFrame
    Train DataFrame
  test: DataFrame
    Test DataFrame

  Returns
  -------
  dataset_hf: DatasetDict
    Dictionary of Hugging Face Datasets
  '''
  
  train_dataset = Dataset.from_pandas(train)
  test_dataset = Dataset.from_pandas(test)
  dataset_hf = DatasetDict()
  
  dataset_hf['train'] = train_dataset
  dataset_hf['test'] = test_dataset

  return dataset_hf

In the following code we create all the data structures that we will need to 
fine-tune our model.
We decided to use as train dataset the union of all the datasets given.
Then, we will predict and evaluate the performances of each model over all the test datasets.

In [ ]:
# Create a dataset composed of all the train datasets
datasets_merged = pd.concat(datasets_train_dict_BIO.values(), ignore_index=True)

# Create of a dictionary that maps all the entity inside the dataset with an incremental id
entity_names = pd.unique(datasets_merged['Entity'])
entity_names_dict = {}
for i, label in enumerate(entity_names):
  entity_names_dict[label] = i

id2label = {}
for i, label in enumerate(entity_names):
  id2label[i] = label

# Creation of all the sentence and the respected labels for the entire dataset
sentences, labels, keys = get_sentences_list_from_df(datasets_merged)
train_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])

# Creation of a DataFrame with all the sentences
for i in range(len(sentences)):
  train_dataset.at[i, 'Tokens'] = sentences[i]
  train_dataset.at[i, 'Labels'] = labels[i]

In [ ]:
# Controll of the possibility to use cuda
device = ('cuda' if torch.cuda.is_available() else 'cpu')

# Download bert models from hugging face
model_name_xxl = 'dbmdz/bert-base-italian-xxl-cased'
model_name_multi = 'bert-base-multilingual-cased'

model_xxl = AutoModelForTokenClassification.from_pretrained(model_name_xxl, num_labels=len(entity_names), label2id=entity_names_dict, id2label=id2label).to(device)
model_multi = AutoModelForTokenClassification.from_pretrained(model_name_multi, num_labels=len(entity_names), label2id=entity_names_dict, id2label=id2label).to(device)

In [ ]:
def tokenize_and_align_labels(examples):

    '''
    Takes a batch of examples. Tokenize the examples and since the bert is performing a sub-tokenization we have to allign the original labels to the actual sub-tokens.

    Parameters
    ----------
    examples: dict
      Dictionary having as values for the key Tokens a list of list of tokenized senteces.

    Returns
    -------
    tokenized_inputs: dict
      dictionary that containt the input_ids created by the bert tokenizer and the respected labels for each token alligned as the initial ones.
  
    '''
    # Tokenization of the sentences
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["Tokens"]), truncation=True, is_split_into_words=True)

    # Alignment of the labels with the original tokens
    labels = []
    for i, label in enumerate(examples['Labels']):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(entity_names_dict[label[word_idx]])
            else:
                label_ids.append(entity_names_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [ ]:
metric = evaluate.load("seqeval")

def compute_metrics(p):

    '''
    It calculates the metrics of a batch of predictions.

    Parameters
    ----------
    p: tuple
      Tuple composed of the predictions by the transformer and the respected original labels.

    Returns
    -------
    entity_metrics: dict
      dictionary of dictionaries that contains for each entity the respected computed metrics

    '''
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    entity_metrics = {}
    for entity_name in entity_names:

        # Take the predictions and the labels of the specific entity (when the original one was that entity)
        true_predictions = [[entity_names[p] for (p, l) in zip(prediction, label) if l != -100 and entity_names[l] == entity_name] for prediction, label in zip(predictions, labels)]
        true_labels = [[entity_names[l] for (p, l) in zip(prediction, label) if l != -100 and entity_names[l] == entity_name] for prediction, label in zip(predictions, labels)]

        # Compute the metrics for that specific entity
        results = metric.compute(predictions=true_predictions, references=true_labels)
        entity_metrics[entity_name] = {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
        }

    return entity_metrics

In [ ]:
def compute_mean_metrics_cv(results):

  '''
  It calculates the metrics of a cross-validation.

  Parameters
  ----------
  results: list
    list that contains all the metrics calculated for each entity every round of the cross-validation.

  Returns
  -------
  metrics: dict
    dictionary of dictionaries that contains for each entity the mean of the metrics.

  '''

  metrics = {}
  for entity_name in entity_names:
    metrics[entity_name] = {
            "precision": sum([scores[f'eval_{entity_name}']['precision'] for scores in results]) / len(results),
            "recall":  sum([scores[f'eval_{entity_name}']['recall'] for scores in results]) / len(results),
            "f1":  sum([scores[f'eval_{entity_name}']['f1'] for scores in results]) / len(results),
            "accuracy":  sum([scores[f'eval_{entity_name}']['accuracy'] for scores in results]) / len(results)
        }
  return metrics

### Cross-Validation
Here we did a cross-validation to find the best hyperparameters for our model.
Calculating the performances of each round and then the mean of all the rounds for that specific pair of hyperparameters.

In [ ]:
learning_rates = [2e-5, 3e-5]
epochs = [2,3,4]

tokenizer = AutoTokenizer.from_pretrained(model_name_xxl)
data_collator = DataCollatorForTokenClassification(tokenizer)

results = []
# Perform of cross-validation
for lr in learning_rates:
  for epoch in epochs:
    results_cv = []
    for i in range(3):    # Cross-validation made of three rounds
      train, val = train_test_split(train_dataset, test_size=0.2, shuffle=True)
      dataset_hf = createDataset(train, val)
      tokenized_datasets = dataset_hf.map(tokenize_and_align_labels, batched=True)

      # Definition of the Transformer's arguments
      args = TrainingArguments(output_dir='ner',
                               evaluation_strategy = "no",
                               learning_rate=lr,
                               per_device_train_batch_size=32,
                               per_device_eval_batch_size=32,
                               num_train_epochs=epoch,
                               weight_decay=0.01,
                               push_to_hub=False,
                               disable_tqdm=False
                              )

      # Creation of the trainer
      trainer = Trainer(
          model_xxl,
          args,
          train_dataset=tokenized_datasets['train'],
          eval_dataset = tokenized_datasets['test'],
          data_collator=data_collator,
          compute_metrics=compute_metrics,
          tokenizer=tokenizer,
      )
      trainer.train()
      results_cv.append(trainer.evaluate())             # Add the matrics for that round to the results of the actual cross-validation
    results.append(compute_mean_metrics_cv(results_cv)) # Calculate the metrics' mean of the cross-validation

### Build the final model
Now, that we found the best hyperparameter, we can train the model over the entire training set.
In this case, first of all, we train the italian model.

In [ ]:
dataset_hf = Dataset.from_pandas(train_dataset)
tokenizer = AutoTokenizer.from_pretrained(model_name_xxl)
tokenized_dataset = dataset_hf.map(tokenize_and_align_labels, batched=True)

data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(output_dir='ner',
                          evaluation_strategy = "no",
                          learning_rate=2e-5,
                          per_device_train_batch_size=32,
                          per_device_eval_batch_size=32,
                          num_train_epochs=4,
                          weight_decay=0.01,
                          push_to_hub=False,
                          disable_tqdm=False
                        )

trainer = Trainer(
    model_xxl,
    args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

We evaluate the model over all the datasets.

In [ ]:
results_test = []
for test in datasets_test_dict_BIO.values():
  sentences, labels, keys = get_sentences_list_from_df(test)
  test_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])
  for i in range(len(sentences)):
    test_dataset.at[i, 'Tokens'] = sentences[i]
    test_dataset.at[i, 'Labels'] = labels[i]
    
  test_dataset = Dataset.from_pandas(test_dataset)
  test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
  results_test.append(trainer.evaluate(test_dataset))

In [ ]:
datasets_test_merged = pd.concat(datasets_test_dict_BIO.values(), ignore_index=True)
sentences, labels, keys = get_sentences_list_from_df(datasets_test_merged)
test_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])
for i in range(len(sentences)):
  test_dataset.at[i, 'Tokens'] = sentences[i]
  test_dataset.at[i, 'Labels'] = labels[i]
test_dataset = Dataset.from_pandas(test_dataset)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
trainer.evaluate(test_dataset)

We repeat the same for the multilingual model.

In [ ]:
dataset_hf = Dataset.from_pandas(train_dataset)
tokenizer = AutoTokenizer.from_pretrained(model_name_multi)
tokenized_dataset = dataset_hf.map(tokenize_and_align_labels, batched=True)

data_collator = DataCollatorForTokenClassification(tokenizer)

args = TrainingArguments(output_dir='ner',
                          evaluation_strategy = "no",
                          learning_rate=2e-5,
                          per_device_train_batch_size=16,
                          per_device_eval_batch_size=16,
                          num_train_epochs=4,
                          weight_decay=0.01,
                          push_to_hub=False,
                          disable_tqdm=False
                        )

trainer = Trainer(
    model_multi,
    args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)
trainer.train()

In [ ]:
results_test = []
for test in datasets_test_dict_BIO.values():
  sentences, labels, keys = get_sentences_list_from_df(test)
  test_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])
  for i in range(len(sentences)):
    test_dataset.at[i, 'Tokens'] = sentences[i]
    test_dataset.at[i, 'Labels'] = labels[i]
    
  test_dataset = Dataset.from_pandas(test_dataset)
  test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
  results_test.append(trainer.evaluate(test_dataset))

In [ ]:
datasets_test_merged = pd.concat(datasets_test_dict_BIO.values(), ignore_index=True)
sentences, labels, keys = get_sentences_list_from_df(datasets_test_merged)
test_dataset = pd.DataFrame(columns=['Tokens', 'Labels'])
for i in range(len(sentences)):
  test_dataset.at[i, 'Tokens'] = sentences[i]
  test_dataset.at[i, 'Labels'] = labels[i]
test_dataset = Dataset.from_pandas(test_dataset)
test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)
trainer.evaluate(test_dataset)

## CODICE ELIA

# KIND DATASET

## Brief Description
KIND (Kessler Italian Named-entities Dataset) is a dataset released in 2022 by researchers from Fondazione Bruno Kessler and the University of Trento. It contains 1 million tokens, of which 600K name-entities are manually annotated. The entities belong to 3 classes (people, location, organization). The texts come from various sources of the Italian language, such as news articles, literature, and political speeches, making it a multi-domain dataset. 
The following table shows the dataset composition:

<table>
  <thead>
    <tr>
      <th rowspan="2">Dataset</th>
      <th rowspan="2">Documents</th>
      <th rowspan="2">Tokens</th>
      <th colspan="4">Train</th>
      <th colspan="4">Test</th>
    </tr>
    <tr>
      <th>Total</th>
      <th>PER</th>
      <th>ORG</th>
      <th>LOC</th>
      <th>Total</th>
      <th>PER</th>
      <th>ORG</th>
      <th>LOC</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <td>Wikinews</td>
      <td>1,000</td>
      <td>308,622</td>
      <td>247,528</td>
      <td>8,928</td>
      <td>7,593</td>
      <td>6,862</td>
      <td>61,094</td>
      <td>1,802</td>
      <td>1,823</td>
      <td>1,711</td>
    </tr>
    <tr>
      <td>Fiction</td>
      <td>86</td>
      <td>192,448</td>
      <td>170,942</td>
      <td>3,439</td>
      <td>182</td>
      <td>733</td>
      <td>21,506</td>
      <td>636</td>
      <td>284</td>
      <td>463</td>
    </tr>
    <tr>
      <td>Aldo Moro</td>
      <td>250</td>
      <td>392,604</td>
      <td>309,798</td>
      <td>1,459</td>
      <td>4,842</td>
      <td>2,024</td>
      <td>82,806</td>
      <td>282</td>
      <td>934</td>
      <td>807</td>
    </tr>
    <tr>
      <td>Alcide De Gasperi</td>
      <td>158</td>
      <td>150,632</td>
      <td>117,997</td>
      <td>1,129</td>
      <td>2,396</td>
      <td>1,046</td>
      <td>32,635</td>
      <td>253</td>
      <td>533</td>
      <td>274</td>
    </tr>
    <tr>
      <td><strong>Total</strong></td>
      <td><strong>1494</strong></td>
      <td><strong>1,044,306</strong></td>
      <td><strong>846,265</strong></td>
      <td><strong>14,955</strong></td>
      <td><strong>15,013</strong></td>
      <td><strong>10,665</strong></td>
      <td><strong>198,041</strong></td>
      <td><strong>2,973</strong></td>
      <td><strong>3,574</strong></td>
      <td><strong>3,255</strong></td>
    </tr>
  </tbody>
</table>


##Annotation Tagging scheme
The tokens of all datasets, except for Aldo Moro, have been manually labeled using the IOB (Inside-Outside-Beginning) convention: each entity is labeled as begin-of-entity (B-[ent]) or continuation-of-entity (I-[ent]). The annotations of the Aldo Moro dataset, instead, were carried out with a mixed process that used both manual and automatic annotations (subsequently checked by hand); due to some differences in the convention for annotation, this dataset does not contain information for composite entities (beginning, continuation). For more details, please refer to the paper related with the dataset release: https://arxiv.org/abs/2112.15099

An example of the annotations is reported here: 


# IOB annotation (from ADG_dev.tsv)
Nel	O
nostro	O
Trentino	B-LOC
attraversiamo	O
un	O
momento	O
storico	O
importante	O
.	O

# non-IOB annotation (from moro_test.tsv)
Dal	O
Consiglio	ORG
nazionale	ORG
del	O
‘	O
75	O
la	O
grande	O
stampa	O
parla	O
di	O
due	O
anime	O
contrapposte	O
del	O
partito	O
.	O

CODICE PER CHARTS. RIMUOVO LE MBRERIE IN SEGUITO

In [ ]:
import pandas as pd
import string
import numpy as np
import matplotlib.pyplot as plt
from math import pi
from matplotlib import rc



BIO_PATH = '../datasets/BIO_tag_NER_notation/'
nonBIO_PATH = '../datasets/Inside_outside_NER_notation/'


SPIDER_ROW_N = 2
SPIDER_COL_N = 2
BAR_ROW_N = 1 
BAR_COL_N = 2

def to_lowerCase(df):
    return pd.DataFrame({'Token': df['Token'].str.lower(), 'Entity': df['Entity']})

def add_column_names(df):
    return  df.rename(columns={0: 'Token', 1: 'Entity'})

def spider_plot(df, group, title, subplot_idx):
    plt.rcParams["figure.figsize"] = (10,10)
    categories=list(df)[:]
    N = len(categories)
    
    # What will be the angle of each axis in the plot? (we divide the plot / number of variable)
    angles = [n / float(N) * 2 * pi for n in range(N)]
    angles += angles[:1]
    
    # Initialise the spider plot
    ax = plt.subplot(SPIDER_ROW_N, SPIDER_COL_N, subplot_idx, polar=True)
    
    # first axis to be on top:
    ax.set_theta_offset(pi / 2)
    ax.set_theta_direction(-1)
    
    # Draw one axe per variable + add labels
    plt.xticks(angles[:-1], categories)
    
    # Draw ylabels
    #ax.set_yscale('log')
    ax.set_rlabel_position(0)
    min = df.min().min()
    max = df.max().max()
    plt.ylim(min -(max-min)/10, max + (max-min)/10)
 
    # Plot each individual = each line of the data
    # I don't make a loop, because plotting more than 3 groups makes the chart unreadable
    
    for i in range(len(group)):
        values=df.loc[i].values.flatten().tolist()
        values += values[:1]
        ax.plot(angles, values, linewidth=1, linestyle='solid', label=group[i])
        ax.fill(angles, values, 'b', alpha=0.1)

    
    # Add legend
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    plt.title(title)

    # Show the graph
    #plt.show()

############################################################### reading datasets ###############################################################

ds = {'ds_mr'           : pd.read_csv(nonBIO_PATH + 'moro_train.tsv', sep='\t', header=None),
      'ds_mr_test'      : pd.read_csv(nonBIO_PATH + 'moro_test.tsv', sep='\t', header=None),
      
      'ds_dg_IOB'       : pd.read_csv(BIO_PATH + 'degasperi_train_BIO.tsv', sep='\t', header=None),
      'ds_fc_IOB'       : pd.read_csv(BIO_PATH + 'fiction_train_BIO.tsv', sep='\t', header=None),
      'ds_wn_IOB'       : pd.read_csv(BIO_PATH + 'wikinews_train_BIO.tsv', sep='\t', header=None),
      'ds_dg_IOB_test'  : pd.read_csv(BIO_PATH + 'degasperi_test_BIO.tsv', sep='\t', header=None),
      'ds_fc_IOB_test'  : pd.read_csv(BIO_PATH + 'fiction_test_BIO.tsv', sep='\t', header=None),
      'ds_wn_IOB_test'  : pd.read_csv(BIO_PATH + 'wikinews_test_BIO.tsv', sep='\t', header=None)
}

############################################################### calculating stats ###############################################################
stats = {}

for i in ds.keys():     
    ds[i] = add_column_names(ds[i])
    ds[i] = to_lowerCase(ds[i])

    stats[i] = {
        'doc_len' : ds[i]['Token'].count(),
        'voc_size' : ds[i]['Token'].nunique(),
        'n_punct': sum(1 for k in ds[i]['Token'] if all(char in string.punctuation for char in k))
    }

    if('IOB' in str(i)):
        
        stats[i]['n_I-PER'] = sum(1 for k in ds[i]['Entity'] if k == 'I-PER') 
        stats[i]['n_I-ORG'] = sum(1 for k in ds[i]['Entity'] if k == 'I-ORG')
        stats[i]['n_I-LOC'] = sum(1 for k in ds[i]['Entity'] if k == 'I-LOC')

        stats[i]['n_B-PER'] = sum(1 for k in ds[i]['Entity'] if k == 'B-PER') 
        stats[i]['n_B-ORG'] = sum(1 for k in ds[i]['Entity'] if k == 'B-ORG')
        stats[i]['n_B-LOC'] = sum(1 for k in ds[i]['Entity'] if k == 'B-LOC')

        stats[i]['n_PER'] = stats[i]['n_I-PER'] + stats[i]['n_B-PER']
        stats[i]['n_ORG'] = stats[i]['n_I-ORG'] + stats[i]['n_B-ORG']
        stats[i]['n_LOC'] = stats[i]['n_I-LOC'] + stats[i]['n_B-LOC']
        
    else:
        stats[i]['n_PER'] = sum(1 for k in ds[i]['Entity'] if k == 'PER') 
        stats[i]['n_ORG'] = sum(1 for k in ds[i]['Entity'] if k == 'ORG')
        stats[i]['n_LOC'] = sum(1 for k in ds[i]['Entity'] if k == 'LOC')

    stats[i]['n_O'] = sum(1 for k in ds[i]['Entity'] if k == 'O')
        

avg_doc_len = sum(stats[i]['doc_len'] for i in stats.keys()) / len(stats.keys())
avg_voc_size = sum(stats[i]['voc_size'] for i in stats.keys()) / len(stats.keys())

############################################################### preparing data for plotting ###############################################################
 
# Values of each group
iper = [stats[i]['n_I-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iorg = [stats[i]['n_I-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iloc = [stats[i]['n_I-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]

bper = [stats[i]['n_B-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
borg = [stats[i]['n_B-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
bloc = [stats[i]['n_B-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]

iob_punct = [stats[i]['n_punct'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iob_o = [stats[i]['n_O'] for i in stats.keys() if 'IOB' in str(i) and 'test' not in str(i)]
iob_o = [iob_o[i] - iob_punct[i] for i in range(len(iob_o))]

iper_test = [stats[i]['n_I-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iorg_test = [stats[i]['n_I-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iloc_test = [stats[i]['n_I-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]

bper_test = [stats[i]['n_B-PER'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
borg_test = [stats[i]['n_B-ORG'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
bloc_test = [stats[i]['n_B-LOC'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]

iob_punct_test = [stats[i]['n_punct'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iob_o_test = [stats[i]['n_O'] for i in stats.keys() if 'IOB' in str(i) and 'test' in str(i)]
iob_o_test = [iob_o_test[i] - iob_punct_test[i] for i in range(len(iob_o_test))]

iob_voc_size = [stats[i]['voc_size'] for i in stats.keys() if 'IOB' in str(i)]

per = [stats[i]['n_PER'] for i in stats.keys() if 'test' not in str(i)]
org = [stats[i]['n_ORG'] for i in stats.keys() if 'test' not in str(i)]
loc = [stats[i]['n_LOC'] for i in stats.keys() if 'test' not in str(i)]

per_test = [stats[i]['n_PER'] for i in stats.keys() if 'test' in str(i)]
org_test = [stats[i]['n_ORG'] for i in stats.keys() if 'test' in str(i)]
loc_test = [stats[i]['n_LOC'] for i in stats.keys() if 'test' in str(i)]

voc_size = [stats[i]['voc_size'] for i in stats.keys() if 'test' not in str(i)]
voc_size_test = [stats[i]['voc_size'] for i in stats.keys() if 'test' in str(i)]

doc_len = [stats[i]['doc_len'] for i in stats.keys() if 'test' not in str(i)]
doc_len_test = [stats[i]['doc_len'] for i in stats.keys() if 'test' in str(i)]

n_punct = [stats[i]['n_punct'] for i in stats.keys() if 'test' not in str(i)]
n_punct_test = [stats[i]['n_punct'] for i in stats.keys() if 'test' in str(i)]

o = [stats[i]['n_O'] for i in stats.keys() if 'test' not in str(i)]
punct = [stats[i]['n_punct'] for i in stats.keys() if 'test' not in str(i)]


o_test = [stats[i]['n_O'] for i in stats.keys() if 'test' in str(i)]
punct_test = [stats[i]['n_punct'] for i in stats.keys() if 'test' in str(i)]

############################################################### spider plots ###############################################################

spider_plot( pd.DataFrame({
    'B-PER': bper,
    'I-PER': iper,
    'B-ORG': borg,
    'I-ORG': iorg,
    'B-LOC': bloc,
    'I-LOC': iloc }),
    ['deGasperi', 'Fiction', 'Wikinews'],
    'Train set IOB tags',
    1)

spider_plot( pd.DataFrame({
    'B-PER': bper_test,
    'I-PER': iper_test,
    'B-ORG': borg_test,
    'I-ORG': iorg_test,
    'B-LOC': bloc_test,
    'I-LOC': iloc_test }),
    ['deGasperi', 'Fiction', 'Wikinews'],
    'Test set IOB tags',
    2)

spider_plot( pd.DataFrame({
    'B-PER': per,
    'B-ORG': org,
    'B-LOC': loc}),
    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
    'Train set non-IOB tags',
    3)

spider_plot( pd.DataFrame({
    'B-PER': per_test,
    'B-ORG': org_test,
    'B-LOC': loc_test}),
    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
    'Test set non-IOB tags',
    4)

#spider_plot( pd.DataFrame({
#    'doc_len': doc_len,
#    'voc_size': voc_size,
#    'n_punct': n_punct,}),
#    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
#    'Train set document statistics',
#    5)
#
#spider_plot( pd.DataFrame({
#    'doc_len': doc_len_test,
#    'voc_size': voc_size_test,
#    'n_punct': n_punct_test,}),
#    ['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
#    'Test set document statistics',
#    6)

plt.show()


def stacked_bar_plot(col_names, data, labels_name, title, subplot_idx):
    plt.rcParams["figure.figsize"] = (15,10)
    # Names of group and bar width
    barWidth = 1
    bars = np.zeros(len(col_names))
    n_col = np.arange(len(col_names))
    plt.subplot(BAR_ROW_N, BAR_COL_N, subplot_idx)


    for i in range(len(data)):
        plt.bar(n_col, data[i], bottom=bars, edgecolor='white', width=barWidth, label=labels_name[i])
        bars = np.add(bars, data[i]).tolist()

    bars = np.add(bper, iper).tolist()
    
    # Custom X axis
    plt.xticks(n_col, col_names, fontweight='bold')
    plt.legend()
    plt.title(title)

    
    # Show graphic
stacked_bar_plot(['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
                 [punct, [o[i]-punct[i] for i in range(len(o))]],
                 ['punct', 'O'],
                 'Train sets',
                 1)

stacked_bar_plot(['Moro', 'deGasperi', 'Fiction', 'Wikinews'],
                 [punct_test, [o_test[i]-punct_test[i] for i in range(len(o_test))]],
                 ['punct', 'O'],
                 'Test sets',
                 2)

plt.show()

In [ ]:
import pandas as pd
import os

#BIO_PATH = 'BIO_tag_NER_notation'
#nonBIO_PATH = 'Inside_outside_NER_notation'

#function to check if 2 tsv files are equal
def check_equal(file1, file2):
	df1 = pd.read_csv(file1, sep='\t')
	df2 = pd.read_csv(file2, sep='\t')
	#printf the path of the two filr, length of the 2 files, and if they are equal
	print(file1, "\n", 
       	  file2, "\n",
		    "equal: ", df1.equals(df2), "\n\n")


#########################à BIO TAGGER ########################################

oTag = "O"  
types = set()

files = {
	"wikinews_train.tsv"	: BIO_PATH + "automatic/WN_train.tsv",
	"wikinews_test.tsv"		: BIO_PATH + "automatic/WN_dev.tsv",
	"fiction_train.tsv"		: BIO_PATH + "automatic/FIC_train.tsv",
	"fiction_test.tsv"		: BIO_PATH + "automatic/FIC_dev.tsv",
	"degasperi_train.tsv"	: BIO_PATH + "automatic/ADG_train.tsv",
	"degasperi_test.tsv"	: BIO_PATH + "automatic/ADG_dev.tsv",
	"moro_train.tsv"		: BIO_PATH + "automatic/MR_train.tsv",
	"moro_test.tsv"			: BIO_PATH + "automatic/MR_dev.tsv",
}

count = {}

for file in files:
	with open(os.path.join(nonBIO_PATH, file), "r") as f:
		outFile = files[file]
		count[outFile] = {"sentences": 0, "tags": {}, "tokens": 0}

		sentences = []
		thisSentence = []

		for line in f:
			line = line.strip()
			if len(line) == 0:
				if len(thisSentence) > 0:
					sentences.append(thisSentence)
					thisSentence = []
				continue
			parts = line.split("\t")
			thisSentence.append(parts)
			count[outFile]["tokens"] += 1

		if len(thisSentence) > 0:
			sentences.append(thisSentence)

		count[outFile]["sentences"] = len(sentences)

		for sentence in sentences:
			previousNer = oTag
			for token in sentence:
				ner = token[1]
				newNer = ner
				if ner != oTag:
					if previousNer != ner:
						if ner not in count[outFile]["tags"]:
							count[outFile]["tags"][ner] = 0
						newNer = "B-" + ner
						count[outFile]["tags"][ner] += 1
						types.add(ner)
					else:
						newNer = "I-" + ner
				token[1] = newNer
				previousNer = ner

		with open(outFile, "w") as fw:
			for sentence in sentences:
				for token in sentence:
					fw.write(token[0])
					fw.write("\t")
					fw.write(token[1])
					fw.write("\n")
				fw.write("\n")

#########################à CHECK ########################################
comp = [
	[BIO_PATH + "automatic/WN_train.tsv", 	'./' + BIO_PATH + '/wikinews_train_BIO.tsv'],
	[BIO_PATH + "automatic/WN_dev.tsv", 	'./' + BIO_PATH + '/wikinews_test_BIO.tsv'],
	[BIO_PATH + "automatic/FIC_train.tsv", 	'./' + BIO_PATH + '/fiction_train_BIO.tsv'],
	[BIO_PATH + "automatic/FIC_dev.tsv", 	'./' + BIO_PATH + '/fiction_test_BIO.tsv'],
	[BIO_PATH + "automatic/ADG_train.tsv", 	'./' + BIO_PATH + '/degasperi_train_BIO.tsv'],
	[BIO_PATH + "automatic/ADG_dev.tsv", 	'./' + BIO_PATH + '/degasperi_test_BIO.tsv'],
	[BIO_PATH + "automatic/MR_train.tsv", 	'./' + BIO_PATH + '/moro_train_BIO.tsv'],
	[BIO_PATH + "automatic/MR_dev.tsv", 	'./' + BIO_PATH + '/moro_test_BIO.tsv'],
]

for i in comp:
	check_equal(i[0], i[1])
